En este proceso se explica la forma en que pueden descargarse los datos disponibles en la [API](https://bchapi-am.developer.azure-api.net/) del Banco Central de Honduras (BCH), utilizando Python.

# Funciones


In [1]:
import urllib.request, json
import pandas as pd
import plotly.express as px
import polars as pl
import subprocess
import os

from IPython.display import Markdown
from tabulate import tabulate

clave_asignada = "xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx" #Favor ingresar la clave proporcionada.

# Las funciones que ejecutan los procesos están guardadas
# en el archivo "/functions/bch_api.py";
# si se ejecuta este código en otra ruta, debe sustituirse el path:
# file_path + "/functions/bch_api.py"
# file_path = os.path.dirname(__file__)
file_path = os.getcwd()
exec(open(file_path + "/functions/bch_api.py").read())

# Acceso a los Datos

Una vez que se crea el registro de usuario (usar el botón "Suscribirse" en el [sitio web](https://bchapi-am.developer.azure-api.net/), puede verse el procedimiento de registro e inicio de sesión en [YouTube](https://www.youtube.com/watch?v=8ZBllMSsKw4)), se necesita obtener una clave (ver explicación en [YouTube](https://www.youtube.com/watch?v=mV90s74OCfc)), ejecutando una consulta al catálogo de indicadores. Favor sustituir en el código la clave actual por su clave asignada.

Para conocer los datos disponibles, se debe ejecutar una consulta al catálogo de indicadores, que contiene información sobre las series disponibles:

* Nombre (código)
* Descripción
* Periodicidad
* Grupo
* Correlativo del Grupo

Como ejemplo de la consulta que contiene todas las variables, se muestran los primeros cinco elementos:


In [2]:
dfmeta = get_groups_vscode()
dfmeta.head()

<string>:29: FutureWarning:

Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.



,Id,Nombre,Descripcion,Periodicidad,Grupo,CorrelativoGrupo
0,1,ESR-PIBT-PROD-DES-CONST-15,Serie Desestacionalizada-VALORES CONSTANTES-21...,Trimestral,ESR-PIBT-PROD-DES-CONST,15
1,2,ESR-PIBT-PROD-DES-CORR-12,Serie Desestacionalizada-VALORES CORRIENTES-14...,Trimestral,ESR-PIBT-PROD-DES-CORR,12
2,3,ESR-PIBT-PROD-DES-CORR-13,Serie Desestacionalizada-VALORES CORRIENTES-16...,Trimestral,ESR-PIBT-PROD-DES-CORR,13
3,4,ESR-PIBT-PROD-DES-CORR-14,Serie Desestacionalizada-VALORES CORRIENTES-20...,Trimestral,ESR-PIBT-PROD-DES-CORR,14
4,5,ESR-PIBT-PROD-DES-CORR-15,Serie Desestacionalizada-VALORES CORRIENTES-21...,Trimestral,ESR-PIBT-PROD-DES-CORR,15


# Grupos

Las variables están categorizadas en 8 grupos principales y 54 subgrupos. La nomenclatura utilizada permite dividir estos en un máximo de cinco niveles, tomando en cuenta el separador (-). Se explicarán los mismos a continuacion, ordenando de acuerdo al número de niveles (número de subgrupos).


In [3]:
# Agregar niveles de Descripción y Grupo
df_all = dfmeta
df_all[['Niv_Descr_1','Niv_Descr_2','Niv_Descr_3','Niv_Descr_4','Niv_Descr_5','Niv_Descr_6','Niv_Descr_7','Niv_Descr_8']] = df_all['Descripcion'].str.split(
    '-',expand=True)
df_all["Niv_Descr"] = df_all.notnull().sum(axis=1) - 6
df_all[
    ['Niv_Gr_1','Niv_Gr_2','Niv_Gr_3','Niv_Gr_4','Niv_Gr_5']]  = df_all['Grupo'].str.split('-',expand=True)
df_all["Niv_Gr"] = df_all.notnull().sum(axis=1) - 2
df_all = pl.DataFrame(df_all)
df_all.write_csv(
        "api/variables.csv",
        separator=";")

res_grupo = save_groups_vscode()
print(res_grupo)

<string>:29: FutureWarning:

Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.



11519 variables
54 grupos


shape: (54, 8)
┌────────────────────┬─────────────┬──────────┬──────────┬──────────┬──────────┬──────────┬────────┐
│ Grupo              ┆ N_Variables ┆ Niv_Gr_1 ┆ Niv_Gr_2 ┆ Niv_Gr_3 ┆ Niv_Gr_4 ┆ Niv_Gr_5 ┆ Niv_Gr │
│ ---                ┆ ---         ┆ ---      ┆ ---      ┆ ---      ┆ ---      ┆ ---      ┆ ---    │
│ str                ┆ i64         ┆ str      ┆ str      ┆ str      ┆ str      ┆ str      ┆ i64    │
╞════════════════════╪═════════════╪══════════╪══════════╪══════════╪══════════╪══════════╪════════╡
│ ESR-PIBT-PROD-OG-C ┆ 17          ┆ ESR      ┆ PIBT     ┆ PROD     ┆ OG       ┆ CORR     ┆ 5      │
│ ORR                ┆             ┆          ┆          ┆          ┆          ┆          ┆        │
│ ESR-PIBT-PROD-DES- ┆ 17          ┆ ESR      ┆ PIBT     ┆ PROD     ┆ DES      ┆ CONST    ┆ 5      │
│ CONST              ┆             ┆          ┆          ┆          ┆          ┆          ┆        │
│ ESR-PIBT-PROD-OG-C ┆ 17          ┆ ESR      ┆ PIBT     ┆ PROD     ┆ OG    

Los 8 grupos mencionados permiten consolidar los archivos de acuerdo con el origen de los datos en la página web del BCH, que en su mayoría pueden consultarse en los [reportes dinámicos](https://www.bch.hn/estadisticas-y-publicaciones-economicas/reportes-dinamicos).

1. EOM = [Estadísticas de Operaciones Monetarias](https://www.bch.hn/estadisticas-y-publicaciones-economicas/reportes-dinamicos/operaciones-de-mercado-abierto)
2. ESR = [Estadísticas del Sector Real](https://www.bch.hn/estadisticas-y-publicaciones-economicas/reportes-dinamicos/sector-real)
3. ESE = [Estadísticas del Sector Externo](https://www.bch.hn/estadisticas-y-publicaciones-economicas/reportes-dinamicos/sector-externo)
4. EMF = [Estadísiticas Monetarias y Financieras (Sector Fiscal)](https://www.bch.hn/estadisticas-y-publicaciones-economicas/reportes-dinamicos/sector-fiscal)
5. EM = [Estadísticas Monetarias (TPM y RIN)](https://www.bch.hn/estadisticas-y-publicaciones-economicas/reportes-dinamicos/sector-monetario)
6. ESP = [Estadísticas de Sistema de Pagos](https://www.bch.hn/estadisticas-y-publicaciones-economicas/reportes-dinamicos/sistema-de-pagos)
7. EC = [Estadísticas de Tipo de Cambio](https://www.bch.hn/estadisticas-y-publicaciones-economicas/reportes-dinamicos/tipo-de-cambio)
8. EP = [Estadísticas de Precios](https://www.bch.hn/estadisticas-y-publicaciones-economicas/reportes-dinamicos/precios)

Los grupos se subdividen en 25 subgrupos:


In [4]:
res = res_nivel_1()
res

<string>:29: FutureWarning:

Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.



11519 variables
54 grupos
54 variables
8 grupos
25 subgrupos


Niv_Gr_1,N_Variables,N_Subgrupos
str,u32,u32
"""ESR""",19,7
"""ESE""",12,6
"""EMF""",12,4
"""EOM""",4,1
"""ESP""",2,2
"""EC""",2,2
"""EM""",2,2
"""EP""",1,1


# Descripción de los Datos

A continuación se presentan los detalles de los grupos por nivel de agregación, ordenados de acuerdo al número de variables que contienen.

### EOM = Estadísticas de Operaciones Monetarias

Los datos están disponibles en la [página web del BCH](https://www.bch.hn/estadisticas-y-publicaciones-economicas/reportes-dinamicos/operaciones-de-mercado-abierto).


In [5]:
res_nivel_2("EOM")

<string>:29: FutureWarning:

Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.



11519 variables
54 grupos


4 variables
1 grupos
4 subgrupos


Niv_Gr_1,Niv_Gr_2,N_Variables,N_Niv_Gr_2
str,str,u32,u32
"""EOM""","""OMA""",4,4


Contiene solamente un subgrupo:

- OMA = Operaciones de Mercado Abierto

#### OMA = Operaciones de Mercado Abierto

Cuatro niveles de agregación:

* 01 = Tasas de Rendimiento Diario de Valores Gubernamentales
* 02 = Créditos (Niv_Descr_4 = 1-3) e Inversiones (Niv_Descr_4 = 4-6): Monto, Número de Solicitudes y Tasa
* 03 = Monto, Número de Operaciones y Tasa
* 04 = Monto, Número de Operaciones y Tasa (Dólares - Lempiras)

En cada una de las tablas subsiguientes, la columna N_Variables corresponde al número de variables que contiene cada nivel.


In [6]:
res_nivel_3("EOM","OMA","Niv_Gr_3").sort("Niv_Gr_3")

4 subgrupos
5575 variables


<string>:29: FutureWarning:

Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.



Niv_Gr_1,Niv_Gr_2,Niv_Gr_3,N_Variables
str,str,str,u32
"""EOM""","""OMA""","""01""",5560
"""EOM""","""OMA""","""02""",6
"""EOM""","""OMA""","""03""",3
"""EOM""","""OMA""","""04""",6


###### 01 = Tasas de Rendimiento Diario de Valores Gubernamentales 

Este es el nivel que contiene más variables, todas corresponden a las "Tasas de Rendimiento Diario de Valores Gubernamentales":


In [7]:
res_nivel_4(
    "EOM","OMA",
    "Niv_Gr_3","01",
    "Niv_Descr_1")

<string>:29: FutureWarning:

Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.



Niv_Gr_1,Niv_Gr_2,Niv_Gr_3,Niv_Descr_1,N_Variables
str,str,str,str,u32
"""EOM""","""OMA""","""01""","""Tasas de Rendi…",5560


Para listar todas las variables:


In [8]:
res_nivel_6(
    "EOM","OMA",
    "Niv_Gr_3","01",
    "Id","Nombre","Niv_Descr_1").sort("Nombre")

<string>:29: FutureWarning:

Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.



Niv_Gr_1,Niv_Gr_2,Niv_Gr_3,Id,Nombre,Niv_Descr_1,N_Variables
str,str,str,i64,str,str,u32
"""EOM""","""OMA""","""01""",701,"""EOM-OMA-01-1""","""Tasas de Rendi…",1
"""EOM""","""OMA""","""01""",710,"""EOM-OMA-01-10""","""Tasas de Rendi…",1
"""EOM""","""OMA""","""01""",800,"""EOM-OMA-01-100…","""Tasas de Rendi…",1
"""EOM""","""OMA""","""01""",1700,"""EOM-OMA-01-100…","""Tasas de Rendi…",1
"""EOM""","""OMA""","""01""",1701,"""EOM-OMA-01-100…","""Tasas de Rendi…",1
…,…,…,…,…,…,…
"""EOM""","""OMA""","""01""",1695,"""EOM-OMA-01-995…","""Tasas de Rendi…",1
"""EOM""","""OMA""","""01""",1696,"""EOM-OMA-01-996…","""Tasas de Rendi…",1
"""EOM""","""OMA""","""01""",1697,"""EOM-OMA-01-997…","""Tasas de Rendi…",1


La consulta para una serie individual puede realizarse solamente mediante el uso de la "Id" correspondiente; por ejemplo, el Id 701 da como resultado:


In [9]:
Id = 1701
df = get_df(Id)
# get_plot(Id)
df

<string>:29: FutureWarning:

Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.



<string>:29: FutureWarning:

Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.



Fecha,Tasas de Rendimiento Diario de Valores Gubernamentales - Dias Plazo - 1001
date,f64
2024-04-19,5.71
2024-04-20,5.71
2024-04-21,5.71
2024-04-22,5.71
2024-04-23,5.71
…,…
2024-05-31,5.8
2024-06-01,5.8
2024-06-02,5.8


###### 02 = Créditos (Niv_Descr_4 = 1-3) e Inversiones (Niv_Descr_4 = 4-6): Monto, Número de Solicitudes y Tasa


In [10]:
res_nivel_4(
    "EOM","OMA",
    "Niv_Gr_3","02",
    "Niv_Descr_5")

<string>:29: FutureWarning:

Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.



Niv_Gr_1,Niv_Gr_2,Niv_Gr_3,Niv_Descr_5,N_Variables
str,str,str,str,u32
"""EOM""","""OMA""","""02""","""Créditos""",3
"""EOM""","""OMA""","""02""","""Inversiones""",3


Todas las variables:


In [11]:
res_nivel_7(
    "EOM","OMA",
    "Niv_Gr_3","02",
    "Id","Nombre","Niv_Descr_5","Niv_Descr_6").sort("Nombre")

<string>:29: FutureWarning:

Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.



Niv_Gr_1,Niv_Gr_2,Niv_Gr_3,Id,Nombre,Niv_Descr_5,Niv_Descr_6,N_Variables
str,str,str,i64,str,str,str,u32
"""EOM""","""OMA""","""02""",6261,"""EOM-OMA-02-1""","""Créditos""","""Monto""",1
"""EOM""","""OMA""","""02""",6262,"""EOM-OMA-02-2""","""Créditos""","""Número de Soli…",1
"""EOM""","""OMA""","""02""",6263,"""EOM-OMA-02-3""","""Créditos""","""Tasa""",1
"""EOM""","""OMA""","""02""",6264,"""EOM-OMA-02-4""","""Inversiones""","""Monto""",1
"""EOM""","""OMA""","""02""",6265,"""EOM-OMA-02-5""","""Inversiones""","""Número de Soli…",1
"""EOM""","""OMA""","""02""",6266,"""EOM-OMA-02-6""","""Inversiones""","""Tasa""",1


Resultados de una sola variable:


In [12]:
Id = 6261
df = get_df(Id)
# get_plot(Id)
df

<string>:29: FutureWarning:

Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.



<string>:29: FutureWarning:

Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.



Fecha,EOM-OMA-02-1-Créditos-Monto
date,f64
2020-01-02,0.0
2020-01-03,0.0
2020-01-06,400.0
2020-01-07,0.0
2020-01-08,0.0
…,…
2024-03-18,168.0
2024-03-19,0.0
2024-03-20,0.0


###### 03 = Monto, Número de Operaciones y Tasa


In [13]:
res_nivel_4(
    "EOM","OMA",
    "Niv_Gr_3","03",
    "Niv_Descr_5")

<string>:29: FutureWarning:

Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.



Niv_Gr_1,Niv_Gr_2,Niv_Gr_3,Niv_Descr_5,N_Variables
str,str,str,str,u32
"""EOM""","""OMA""","""03""","""Tasa""",1
"""EOM""","""OMA""","""03""","""Monto""",1
"""EOM""","""OMA""","""03""","""Número de Oper…",1


Todas las variables:


In [14]:
res_nivel_6(
    "EOM","OMA",
    "Niv_Gr_3","03",
    "Id","Nombre","Niv_Descr_5").sort("Nombre")

<string>:29: FutureWarning:

Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.



Niv_Gr_1,Niv_Gr_2,Niv_Gr_3,Id,Nombre,Niv_Descr_5,N_Variables
str,str,str,i64,str,str,u32
"""EOM""","""OMA""","""03""",6267,"""EOM-OMA-03-1""","""Monto""",1
"""EOM""","""OMA""","""03""",6268,"""EOM-OMA-03-2""","""Número de Oper…",1
"""EOM""","""OMA""","""03""",6269,"""EOM-OMA-03-3""","""Tasa""",1


Resultados de una sola variable:


In [15]:
Id = 6267
df = get_df(Id)
# get_plot(Id)
df

<string>:29: FutureWarning:

Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.



<string>:29: FutureWarning:

Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.



Fecha,EOM-OMA-03-1-Monto
date,f64
2019-01-02,1347.0
2019-01-03,0.0
2019-01-04,400.0
2019-01-07,3857.1
2019-01-08,0.0
…,…
2024-03-18,1500.0
2024-03-19,1044.0
2024-03-20,297.5


###### 04 = Monto, Número de Operaciones y Tasa (Dólares - Lempiras)


In [16]:
res_nivel_4(
    "EOM","OMA",
    "Niv_Gr_3",
    "04",
    "Niv_Descr_5")

<string>:29: FutureWarning:

Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.



Niv_Gr_1,Niv_Gr_2,Niv_Gr_3,Niv_Descr_5,N_Variables
str,str,str,str,u32
"""EOM""","""OMA""","""04""","""Tasa""",2
"""EOM""","""OMA""","""04""","""Monto""",2
"""EOM""","""OMA""","""04""","""Número de Oper…",2


In [17]:
res_nivel_5(
    "EOM","OMA",
    "Niv_Gr_3","04",
    "Niv_Descr_5","Niv_Descr_6")

<string>:29: FutureWarning:

Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.



Niv_Gr_1,Niv_Gr_2,Niv_Gr_3,Niv_Descr_5,Niv_Descr_6,N_Variables
str,str,str,str,str,u32
"""EOM""","""OMA""","""04""","""Tasa""","""Lempiras""",1
"""EOM""","""OMA""","""04""","""Monto""","""Dólares""",1
"""EOM""","""OMA""","""04""","""Número de Oper…","""Dólares""",1
"""EOM""","""OMA""","""04""","""Monto""","""Lempiras""",1
"""EOM""","""OMA""","""04""","""Número de Oper…","""Lempiras""",1
"""EOM""","""OMA""","""04""","""Tasa""","""Dólares""",1


Todas las variables:


In [18]:
res_nivel_7(
    "EOM","OMA",
    "Niv_Gr_3","04",
    "Id","Nombre","Niv_Descr_5","Niv_Descr_6").sort("Nombre")

<string>:29: FutureWarning:

Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.



Niv_Gr_1,Niv_Gr_2,Niv_Gr_3,Id,Nombre,Niv_Descr_5,Niv_Descr_6,N_Variables
str,str,str,i64,str,str,str,u32
"""EOM""","""OMA""","""04""",6270,"""EOM-OMA-04-1""","""Monto""","""Dólares""",1
"""EOM""","""OMA""","""04""",6271,"""EOM-OMA-04-2""","""Monto""","""Lempiras""",1
"""EOM""","""OMA""","""04""",6272,"""EOM-OMA-04-3""","""Número de Oper…","""Dólares""",1
"""EOM""","""OMA""","""04""",6273,"""EOM-OMA-04-4""","""Número de Oper…","""Lempiras""",1
"""EOM""","""OMA""","""04""",6274,"""EOM-OMA-04-5""","""Tasa""","""Dólares""",1
"""EOM""","""OMA""","""04""",6275,"""EOM-OMA-04-6""","""Tasa""","""Lempiras""",1


Resultados de una sola variable:


In [19]:
Id = 6270
df = get_df(Id)
# get_plot(Id)
df

<string>:29: FutureWarning:

Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.



<string>:29: FutureWarning:

Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.



Fecha,EOM-OMA-04-1-Monto-Dólares
date,f64
2019-01-02,0.0
2019-01-03,0.0
2019-01-04,0.0
2019-01-07,0.0
2019-01-08,0.0
…,…
2024-03-18,0.0
2024-03-19,0.0
2024-03-20,0.0


### ESR = Estadísticas del Sector Real


In [20]:
res_nivel_2("ESR")

11519 variables
54 grupos
19 variables
7 grupos
13 subgrupos


<string>:29: FutureWarning:

Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.



Niv_Gr_1,Niv_Gr_2,N_Variables,N_Niv_Gr_2
str,str,u32,u32
"""ESR""","""PIBT""",9,3
"""ESR""","""PIBA""",4,4
"""ESR""","""VAB""",2,2
"""ESR""","""ODA""",1,1
"""ESR""","""IMAE""",1,1
"""ESR""","""CCI""",1,1
"""ESR""","""COU""",1,1


* COU = Cuadro de Oferta y Utilización
* CCI = Clasificación Cruzada Industria
* VAB = Valor Agregado Bruto
* PIBA = PIB Anual
* PIBT = PIB Trimestral
* ODA = Oferta y Demanda Agregada
* IMAE = Índice Mensual de la Actividad Económica

#### COU = Cuadro de Oferta y Utilización


In [21]:
res_nivel_3("ESR","COU","Niv_Gr_3")

1 subgrupos
3608 variables


<string>:29: FutureWarning:

Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.



Niv_Gr_1,Niv_Gr_2,Niv_Gr_3,N_Variables
str,str,str,u32
"""ESR""","""COU""","""01""",3608


Detalla la interacción del producto con la actividad (el detalle se toma de acuerdo con la estructura de la columna "Nombre"). Para conocer la estructura de las estadísticas derivadas de este cuadro, consultar en la [página web del BCH](https://www.bch.hn/estadisticas-y-publicaciones-economicas/sector-real/cuentas-nacionales-anuales-base-2000/cuadro-oferta-y-utilizacion).

El orden está de acuerdo con el Nivel de Descripción 2 (Niv_Descr_2) y Nivel de Descripción 3 (Niv_Descr_3) de la columna Nombre, para los casos de:

* VALOR BRUTO DE PRODUCCIÓN
* CONSUMO INTERMEDIO
* VALOR AGREGADO
* OFERTA
* UTILIZACIÓN

Cada uno de estos detalles corresponde a una hoja de Excel en el archivo anual correspondiente.


In [22]:
res_nivel_4(
    "ESR","COU",
    "Niv_Gr_3",
    "01",
    "Niv_Descr_1")

<string>:29: FutureWarning:

Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.



Niv_Gr_1,Niv_Gr_2,Niv_Gr_3,Niv_Descr_1,N_Variables
str,str,str,str,u32
"""ESR""","""COU""","""01""","""VALOR BRUTO DE…",1044
"""ESR""","""COU""","""01""","""CONSUMO INTERM…",1044
"""ESR""","""COU""","""01""","""OFERTA""",580
"""ESR""","""COU""","""01""","""UTILIZACIÓN""",580
"""ESR""","""COU""","""01""","""VALOR AGREGADO…",360


##### VALOR BRUTO DE PRODUCCIÓN

En el cuadro original, las filas son los productos a un nivel de dos dígitos de la Nomenclatura de Productos de Honduras (NPH) y las columnas son las actiivades económicas tomando en cuenta la Nomenclatura de Actividades Económicas de Honduras (NAEH).


In [23]:
res_nivel_7(
    "ESR","COU",
    "Niv_Gr_3",
    "01",
    "Niv_Descr_1","Id","Nombre","Descripcion").filter(pl.col("Niv_Descr_1") == "VALOR BRUTO DE PRODUCCIÓN")

<string>:29: FutureWarning:

Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.



Niv_Gr_1,Niv_Gr_2,Niv_Gr_3,Niv_Descr_1,Id,Nombre,Descripcion,N_Variables
str,str,str,str,i64,str,str,u32
"""ESR""","""COU""","""01""","""VALOR BRUTO DE…",10386,"""ESR-COU-01-256…","""VALOR BRUTO DE…",1
"""ESR""","""COU""","""01""","""VALOR BRUTO DE…",10395,"""ESR-COU-01-257…","""VALOR BRUTO DE…",1
"""ESR""","""COU""","""01""","""VALOR BRUTO DE…",10403,"""ESR-COU-01-258…","""VALOR BRUTO DE…",1
"""ESR""","""COU""","""01""","""VALOR BRUTO DE…",10405,"""ESR-COU-01-258…","""VALOR BRUTO DE…",1
"""ESR""","""COU""","""01""","""VALOR BRUTO DE…",10435,"""ESR-COU-01-261…","""VALOR BRUTO DE…",1
…,…,…,…,…,…,…,…
"""ESR""","""COU""","""01""","""VALOR BRUTO DE…",11309,"""ESR-COU-01-349…","""VALOR BRUTO DE…",1
"""ESR""","""COU""","""01""","""VALOR BRUTO DE…",11321,"""ESR-COU-01-350…","""VALOR BRUTO DE…",1
"""ESR""","""COU""","""01""","""VALOR BRUTO DE…",11325,"""ESR-COU-01-350…","""VALOR BRUTO DE…",1


Resultados de una sola variable:


In [24]:
Id = 10410
df = get_df(Id)
# get_plot(Id)
df

<string>:29: FutureWarning:

Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.



<string>:29: FutureWarning:

Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.



Fecha,VALOR BRUTO DE PRODUCCIÓN-ANIMALES VIVOS-INTERMEDIACIÓN FINANCIERA
date,i64
2000-01-01,0
2001-01-01,0
2002-01-01,0
2003-01-01,0
2004-01-01,0
…,…
2018-01-01,0
2019-01-01,0
2020-01-01,0


##### CONSUMO INTERMEDIO

En el cuadro original, las filas son los productos a un nivel de dos dígitos de la Nomenclatura de Productos de Honduras (NPH) y las columnas son las actiivades económicas tomando en cuenta la Nomenclatura de Actividades Económicas de Honduras (NAEH).


In [25]:
res_nivel_7(
    "ESR","COU",
    "Niv_Gr_3",
    "01",
    "Niv_Descr_1","Id","Nombre","Descripcion").filter(pl.col("Niv_Descr_1") == "CONSUMO INTERMEDIO")

<string>:29: FutureWarning:

Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.



Niv_Gr_1,Niv_Gr_2,Niv_Gr_3,Niv_Descr_1,Id,Nombre,Descripcion,N_Variables
str,str,str,str,i64,str,str,u32
"""ESR""","""COU""","""01""","""CONSUMO INTERM…",7838,"""ESR-COU-01-21""","""CONSUMO INTERM…",1
"""ESR""","""COU""","""01""","""CONSUMO INTERM…",7866,"""ESR-COU-01-49""","""CONSUMO INTERM…",1
"""ESR""","""COU""","""01""","""CONSUMO INTERM…",7874,"""ESR-COU-01-57""","""CONSUMO INTERM…",1
"""ESR""","""COU""","""01""","""CONSUMO INTERM…",7875,"""ESR-COU-01-58""","""CONSUMO INTERM…",1
"""ESR""","""COU""","""01""","""CONSUMO INTERM…",7879,"""ESR-COU-01-62""","""CONSUMO INTERM…",1
…,…,…,…,…,…,…,…
"""ESR""","""COU""","""01""","""CONSUMO INTERM…",8727,"""ESR-COU-01-910…","""CONSUMO INTERM…",1
"""ESR""","""COU""","""01""","""CONSUMO INTERM…",8788,"""ESR-COU-01-971…","""CONSUMO INTERM…",1
"""ESR""","""COU""","""01""","""CONSUMO INTERM…",8793,"""ESR-COU-01-976…","""CONSUMO INTERM…",1


Resultados de una sola variable:


In [26]:
Id = 7835
df = get_df(Id)
# get_plot(Id)
df

<string>:29: FutureWarning:

Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.



<string>:29: FutureWarning:

Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.



Fecha,CONSUMO INTERMEDIO-ACEITES Y GRASAS ANIMALES Y VEGETALES-TRANSPORTE Y ALMACENAMIENTO
date,i64
2000-01-01,0
2001-01-01,0
2002-01-01,0
2003-01-01,0
2004-01-01,0
…,…
2018-01-01,0
2019-01-01,0
2020-01-01,0


##### VALOR AGREGADO

En el cuadro original, las filas son los elementos indispensables para calcular el Valor Agregado Bruto, VAB (Remuneración de los Asalariados, Impuestos sobre la Producción y las Importaciones, Subvenciones, Excedente de Explotación, Ingreso Mixto y Consumo de Capital Fijo) y las columnas son las actiivades económicas tomando en cuenta la Nomenclatura de Actividades Económicas de Honduras (NAEH).


In [27]:
res_nivel_7(
    "ESR","COU",
    "Niv_Gr_3",
    "01",
    "Niv_Descr_1","Id","Nombre","Descripcion").filter(pl.col("Niv_Descr_1") == "VALOR AGREGADO")

<string>:29: FutureWarning:

Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.



Niv_Gr_1,Niv_Gr_2,Niv_Gr_3,Niv_Descr_1,Id,Nombre,Descripcion,N_Variables
str,str,str,str,i64,str,str,u32
"""ESR""","""COU""","""01""","""VALOR AGREGADO…",10030,"""ESR-COU-01-221…","""VALOR AGREGADO…",1
"""ESR""","""COU""","""01""","""VALOR AGREGADO…",10124,"""ESR-COU-01-230…","""VALOR AGREGADO…",1
"""ESR""","""COU""","""01""","""VALOR AGREGADO…",10134,"""ESR-COU-01-231…","""VALOR AGREGADO…",1
"""ESR""","""COU""","""01""","""VALOR AGREGADO…",10148,"""ESR-COU-01-233…","""VALOR AGREGADO…",1
"""ESR""","""COU""","""01""","""VALOR AGREGADO…",10153,"""ESR-COU-01-233…","""VALOR AGREGADO…",1
…,…,…,…,…,…,…,…
"""ESR""","""COU""","""01""","""VALOR AGREGADO…",10291,"""ESR-COU-01-247…","""VALOR AGREGADO…",1
"""ESR""","""COU""","""01""","""VALOR AGREGADO…",10303,"""ESR-COU-01-248…","""VALOR AGREGADO…",1
"""ESR""","""COU""","""01""","""VALOR AGREGADO…",10330,"""ESR-COU-01-251…","""VALOR AGREGADO…",1


Resultados de una sola variable:


In [28]:
Id = 10024
df = get_df(Id)
# get_plot(Id)
df

<string>:29: FutureWarning:

Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.



<string>:29: FutureWarning:

Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.



Fecha,"VALOR AGREGADO-CONSUMO DE CAPITAL FIJO NO SOCIEDADES-AGRICULTURA, GANADERIA, CAZA, SILVICULTURA Y PESCA"
date,f64
2000-01-01,502.5586
2001-01-01,99.3346
2002-01-01,111.2838
2003-01-01,118.6011
2004-01-01,119.6569
…,…
2018-01-01,180.6
2019-01-01,157.9
2020-01-01,152.9


##### OFERTA

En el cuadro original, las filas son los productos a un nivel de dos dígitos de la Nomenclatura de Productos de Honduras (NPH) y las columnas son los componentes de la Oferta Agregada a Precios de Comprador (Producción, Importaciones CIF, Ajuste CIF/FOB, Impuestos y Subvenciones a los Productos y Márgenes de Distribución).


In [29]:
res_nivel_7(
    "ESR","COU",
    "Niv_Gr_3",
    "01",
    "Niv_Descr_1","Id","Nombre","Descripcion").filter(pl.col("Niv_Descr_1") == "OFERTA")

<string>:29: FutureWarning:

Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.



Niv_Gr_1,Niv_Gr_2,Niv_Gr_3,Niv_Descr_1,Id,Nombre,Descripcion,N_Variables
str,str,str,str,i64,str,str,u32
"""ESR""","""COU""","""01""","""OFERTA""",8884,"""ESR-COU-01-106…","""OFERTA-AZÚCAR …",1
"""ESR""","""COU""","""01""","""OFERTA""",8916,"""ESR-COU-01-109…","""OFERTA-CACAO, …",1
"""ESR""","""COU""","""01""","""OFERTA""",8918,"""ESR-COU-01-110…","""OFERTA-CACAO, …",1
"""ESR""","""COU""","""01""","""OFERTA""",8932,"""ESR-COU-01-111…","""OFERTA-CARNE Y…",1
"""ESR""","""COU""","""01""","""OFERTA""",8966,"""ESR-COU-01-114…","""OFERTA-CONSTRU…",1
…,…,…,…,…,…,…,…
"""ESR""","""COU""","""01""","""OFERTA""",9391,"""ESR-COU-01-157…","""OFERTA-TABACO …",1
"""ESR""","""COU""","""01""","""OFERTA""",9392,"""ESR-COU-01-157…","""OFERTA-TEXTILE…",1
"""ESR""","""COU""","""01""","""OFERTA""",9415,"""ESR-COU-01-159…","""OFERTA-TRIGO-M…",1


Resultados de una sola variable:


In [30]:
Id = 8878
df = get_df(Id)
# get_plot(Id)
df

<string>:29: FutureWarning:

Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.



<string>:29: FutureWarning:

Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.



Fecha,OFERTA-ANIMALES VIVOS-TOTAL
date,f64
2000-01-01,51.0864
2001-01-01,53.1902
2002-01-01,55.1843
2003-01-01,55.0125
2004-01-01,57.6104
…,…
2018-01-01,216.1
2019-01-01,223.0
2020-01-01,202.3


##### UTILIZACIÓN

En el cuadro original, las filas son los productos a un nivel de dos dígitos de la Nomenclatura de Productos de Honduras (NPH) y las columnas son los componentes de la Utilización Agregada a Precios de Comprador (Consumo Intermedio, Exportciones FOB, Gasto de Consumo Final y Formación Bruta de Capital).


In [31]:
res_nivel_7(
    "ESR","COU",
    "Niv_Gr_3",
    "01",
    "Niv_Descr_1","Id","Nombre","Descripcion").filter(pl.col("Niv_Descr_1") == "UTILIZACIÓN")

<string>:29: FutureWarning:

Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.



Niv_Gr_1,Niv_Gr_2,Niv_Gr_3,Niv_Descr_1,Id,Nombre,Descripcion,N_Variables
str,str,str,str,i64,str,str,u32
"""ESR""","""COU""","""01""","""UTILIZACIÓN""",9474,"""ESR-COU-01-165…","""UTILIZACIÓN-BA…",1
"""ESR""","""COU""","""01""","""UTILIZACIÓN""",9505,"""ESR-COU-01-168…","""UTILIZACIÓN-CA…",1
"""ESR""","""COU""","""01""","""UTILIZACIÓN""",9543,"""ESR-COU-01-172…","""UTILIZACIÓN-CO…",1
"""ESR""","""COU""","""01""","""UTILIZACIÓN""",9548,"""ESR-COU-01-173…","""UTILIZACIÓN-CO…",1
"""ESR""","""COU""","""01""","""UTILIZACIÓN""",9549,"""ESR-COU-01-173…","""UTILIZACIÓN-CO…",1
…,…,…,…,…,…,…,…
"""ESR""","""COU""","""01""","""UTILIZACIÓN""",9948,"""ESR-COU-01-213…","""UTILIZACIÓN-SE…",1
"""ESR""","""COU""","""01""","""UTILIZACIÓN""",9958,"""ESR-COU-01-214…","""UTILIZACIÓN-SU…",1
"""ESR""","""COU""","""01""","""UTILIZACIÓN""",9987,"""ESR-COU-01-217…","""UTILIZACIÓN-TO…",1


Tomar en cuenta que, para realizar el cálculo del VAB por NAEH, se debe sumar las columnas de Valor Bruto de Producción y restar las de Consumo Intermedio; adicionalmente, como los datos correspondientes a Impuestos sobre los Productos y los Márgenes de Distribución solamente se encuentran a nivel de NPH, no se puede tener un VAB total que se obtenga por suma de VAB a nivel de actividad individual, en vista que debe restarse las de Subvenciones a los productos, mismas que se encuentran en el Cuadro de Oferta solamente a nivel de NPH. Esta forma de obtener el VAB se conoce como el cálculo del PIB por el enfoque del Ingreso.

Como la Oferta y Utilización están detalladas a nivel de NPH, el VAB por NPH a partir de los dos detalles previos, se puede obtener por dos métodos:

1. Suma de Producción + Impuestos - Subvenciones a los Productos + Márgenes de Distribución (Oferta) - Consumo Intermedio (Utilización) => PIB por el enfoque de la Produccion.
2. Exportaciones FOB + Gasto de Consumo Final + Formación Bruta de Capital (Utilización) - Importaciones CIF - Ajuste CIF/FOB (Oferta) => PIB por el enfoque del Gasto.

Para saber el contenido de todas las variables:


In [32]:
res_nivel_7(
    "ESR","COU",
    "Niv_Gr_3",
    "01",
    "Id","Niv_Descr_1","Nombre","Descripcion").sort("Id")

<string>:29: FutureWarning:

Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.



Niv_Gr_1,Niv_Gr_2,Niv_Gr_3,Id,Niv_Descr_1,Nombre,Descripcion,N_Variables
str,str,str,i64,str,str,str,u32
"""ESR""","""COU""","""01""",7818,"""CONSUMO INTERM…","""ESR-COU-01-1""","""CONSUMO INTERM…",1
"""ESR""","""COU""","""01""",7819,"""CONSUMO INTERM…","""ESR-COU-01-2""","""CONSUMO INTERM…",1
"""ESR""","""COU""","""01""",7820,"""CONSUMO INTERM…","""ESR-COU-01-3""","""CONSUMO INTERM…",1
"""ESR""","""COU""","""01""",7821,"""CONSUMO INTERM…","""ESR-COU-01-4""","""CONSUMO INTERM…",1
"""ESR""","""COU""","""01""",7822,"""CONSUMO INTERM…","""ESR-COU-01-5""","""CONSUMO INTERM…",1
…,…,…,…,…,…,…,…
"""ESR""","""COU""","""01""",11421,"""VALOR BRUTO DE…","""ESR-COU-01-360…","""VALOR BRUTO DE…",1
"""ESR""","""COU""","""01""",11422,"""VALOR BRUTO DE…","""ESR-COU-01-360…","""VALOR BRUTO DE…",1
"""ESR""","""COU""","""01""",11423,"""VALOR BRUTO DE…","""ESR-COU-01-360…","""VALOR BRUTO DE…",1


Resultados de una sola variable:


In [33]:
Id = 9465
df = get_df(Id)
# get_plot(Id)
df

<string>:29: FutureWarning:

Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.



<string>:29: FutureWarning:

Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.



Fecha,UTILIZACIÓN-AZÚCAR Y MELAZAS-GASTO DE CONSUMO FINAL HOGARES
date,f64
2000-01-01,1123.2441
2001-01-01,1205.4408
2002-01-01,1273.3699
2003-01-01,1462.889
2004-01-01,1641.5371
…,…
2018-01-01,4614.6
2019-01-01,4790.1
2020-01-01,4047.3


#### CCI = Clasificación Cruzada Industria Sector Institucional

Para conocer la estructura de las estadísticas derivadas de este cuadro, consultar en la [página web del BCH](https://www.bch.hn/estadisticas-y-publicaciones-economicas/sector-real/cuentas-nacionales-anuales-base-2000/clasificacion-cruzada-industria-sector-institucional).

El archivo contiene 3 hojas, que pueden filtrarse de acuerdo con el Nivel de Descripción 1 (Niv_Descr_1) de la variable:

* Producción Bruta
* Consumo Intermedio
* Valor Agregado

Los cuadros en cada hoja contienen los valores a nivel de Industria en las filas y a nivel de Sector Institucional en las columnas.

Debido a que el VAB implica tomar en cuenta impuestos y subvenciones a nivel de NPH, no es posible obtener el cálculo de esta variable por suma a partir de los cuadros del CCI (ver explicación en el apartado del COU).


In [34]:
res_nivel_3("ESR","CCI","Niv_Descr_1")

3 subgrupos
371 variables


<string>:29: FutureWarning:

Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.



Niv_Gr_1,Niv_Gr_2,Niv_Descr_1,N_Variables
str,str,str,u32
"""ESR""","""CCI""","""Consumo Interm…",126
"""ESR""","""CCI""","""Valor Agregado…",126
"""ESR""","""CCI""","""Producción""",119


"Niv_Descr_2" es la Industria (filas).


In [35]:
res_nivel_5(
    "ESR","CCI",
    "Niv_Gr_3",
    "01",
    "Niv_Descr_1","Niv_Descr_2")

<string>:29: FutureWarning:

Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.



Niv_Gr_1,Niv_Gr_2,Niv_Gr_3,Niv_Descr_1,Niv_Descr_2,N_Variables
str,str,str,str,str,u32
"""ESR""","""CCI""","""01""","""Producción""","""Servicios de E…",7
"""ESR""","""CCI""","""01""","""Producción""","""Servicios Soci…",7
"""ESR""","""CCI""","""01""","""Consumo Interm…","""Industrias Man…",7
"""ESR""","""CCI""","""01""","""Consumo Interm…","""Servicios de E…",7
"""ESR""","""CCI""","""01""","""Producción""","""Industrias Man…",7
…,…,…,…,…,…
"""ESR""","""CCI""","""01""","""Valor Agregado…","""Industrias Man…",7
"""ESR""","""CCI""","""01""","""Valor Agregado…","""Servicios de I…",7
"""ESR""","""CCI""","""01""","""Consumo Interm…","""Construcción""",7


"Niv_Descr_3" es el Sector Institucional (columnas).


In [36]:
res_nivel_5(
    "ESR","CCI",
    "Niv_Gr_3",
    "01",
    "Niv_Descr_1","Niv_Descr_3")

<string>:29: FutureWarning:

Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.



Niv_Gr_1,Niv_Gr_2,Niv_Gr_3,Niv_Descr_1,Niv_Descr_3,N_Variables
str,str,str,str,str,u32
"""ESR""","""CCI""","""01""","""Valor Agregado…","""Hogares""",18
"""ESR""","""CCI""","""01""","""Consumo Interm…","""Gobierno Gener…",18
"""ESR""","""CCI""","""01""","""Valor Agregado…","""SIFMI""",18
"""ESR""","""CCI""","""01""","""Consumo Interm…","""Sociedades Fi…",18
"""ESR""","""CCI""","""01""","""Consumo Interm…","""Sociedades No…",18
…,…,…,…,…,…
"""ESR""","""CCI""","""01""","""Producción""","""ISFLSH""",17
"""ESR""","""CCI""","""01""","""Producción""","""Sociedades Fi…",17
"""ESR""","""CCI""","""01""","""Producción""","""Sociedades No…",17


Para saber el contenido de todas las variables:


In [37]:
res_nivel_7(
    "ESR","CCI",
    "Niv_Gr_3",
    "01",
    "Id","Niv_Descr_1","Niv_Descr_2","Niv_Descr_3")

<string>:29: FutureWarning:

Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.



Niv_Gr_1,Niv_Gr_2,Niv_Gr_3,Id,Niv_Descr_1,Niv_Descr_2,Niv_Descr_3,N_Variables
str,str,str,i64,str,str,str,u32
"""ESR""","""CCI""","""01""",7448,"""Consumo Interm…","""Actividades In…","""Gobierno Gener…",1
"""ESR""","""CCI""","""01""",7469,"""Consumo Interm…","""Comercio, Repa…","""Gobierno Gener…",1
"""ESR""","""CCI""","""01""",7492,"""Consumo Interm…","""Electricidad y…","""ISFLSH""",1
"""ESR""","""CCI""","""01""",7494,"""Consumo Interm…","""Electricidad y…","""Sociedades Fi…",1
"""ESR""","""CCI""","""01""",7539,"""Consumo Interm…","""Servicios de E…","""Gobierno Gener…",1
…,…,…,…,…,…,…,…
"""ESR""","""CCI""","""01""",7652,"""Producción""","""Propiedad de V…","""Hogares""",1
"""ESR""","""CCI""","""01""",7665,"""Producción""","""Servicios de E…","""Gobierno Gener…",1
"""ESR""","""CCI""","""01""",7727,"""Valor Agregado…","""Construcción""","""Economía Total…",1


Resultados de una sola variable:


In [38]:
Id = 7467
df = get_df(Id)
# get_plot(Id)
df

<string>:29: FutureWarning:

Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.



<string>:29: FutureWarning:

Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.



Fecha,"Consumo Intermedio-Agricultura, Ganadería, Caza, Silvicultura y Pesca-Sociedades No Financieras"
date,f64
2000-01-01,4310.7
2001-01-01,4789.7977
2002-01-01,5304.4434
2003-01-01,6281.5443
2004-01-01,7631.9981
…,…
2015-01-01,19002.0
2016-01-01,19951.6
2017-01-01,21930.4


#### VAB = Valor Agregado Bruto


In [39]:
res_nivel_3("ESR","VAB","Niv_Gr_3")

2 subgrupos
120 variables


<string>:29: FutureWarning:

Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.



Niv_Gr_1,Niv_Gr_2,Niv_Gr_3,N_Variables
str,str,str,u32
"""ESR""","""VAB""","""02""",64
"""ESR""","""VAB""","""01""",56


##### Valor y Variación, Corrientes y Constantes, por NAEH


In [40]:
res_nivel_4(
    "ESR","VAB",
    "Niv_Gr_3",
    "01",
    "Niv_Descr_4")

<string>:29: FutureWarning:

Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.



Niv_Gr_1,Niv_Gr_2,Niv_Gr_3,Niv_Descr_4,N_Variables
str,str,str,str,u32
"""ESR""","""VAB""","""01""",""" Variación """,28
"""ESR""","""VAB""","""01""",""" Valor """,28


In [41]:
res_nivel_5(
    "ESR","VAB",
    "Niv_Gr_3",
    "01",
    "Niv_Descr_4","Niv_Descr_5")

<string>:29: FutureWarning:

Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.



Niv_Gr_1,Niv_Gr_2,Niv_Gr_3,Niv_Descr_4,Niv_Descr_5,N_Variables
str,str,str,str,str,u32
"""ESR""","""VAB""","""01""",""" Variación """,""" Corrientes """,14
"""ESR""","""VAB""","""01""",""" Valor """,""" Constante """,14
"""ESR""","""VAB""","""01""",""" Variación """,""" Constante """,14
"""ESR""","""VAB""","""01""",""" Valor """,""" Corrientes """,14


Para saber el contenido de todas las variables:


In [42]:
res_nivel_7(
    "ESR","VAB",
    "Niv_Gr_3",
    "01",
    "Id","Niv_Descr_4","Niv_Descr_5","Niv_Descr_6")

<string>:29: FutureWarning:

Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.



Niv_Gr_1,Niv_Gr_2,Niv_Gr_3,Id,Niv_Descr_4,Niv_Descr_5,Niv_Descr_6,N_Variables
str,str,str,i64,str,str,str,u32
"""ESR""","""VAB""","""01""",222,""" Valor """,""" Corrientes """,""" Cultivo de Ba…",1
"""ESR""","""VAB""","""01""",225,""" Valor """,""" Corrientes """,""" Cría de Ganad…",1
"""ESR""","""VAB""","""01""",243,""" Valor """,""" Constante """,""" Silvicultura""",1
"""ESR""","""VAB""","""01""",254,""" Variación """,""" Corrientes """,""" Actividades d…",1
"""ESR""","""VAB""","""01""",229,""" Valor """,""" Corrientes """,""" Silvicultura""",1
…,…,…,…,…,…,…,…
"""ESR""","""VAB""","""01""",227,""" Valor """,""" Corrientes """,""" Cría de Otros…",1
"""ESR""","""VAB""","""01""",235,""" Valor """,""" Constante """,""" Cultivo de Pa…",1
"""ESR""","""VAB""","""01""",249,""" Variación """,""" Corrientes """,""" Cultivo de Pa…",1


Resultados de una sola variable:


In [43]:
Id = 220
df = get_df(Id)
# get_plot(Id)
df

<string>:29: FutureWarning:

Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.



<string>:29: FutureWarning:

Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.



Fecha,ESR-VAB-01 - Valor - Corrientes - Cultivo de Arroz en Granza
date,f64
2000-01-01,18.7
2001-01-01,19.9
2002-01-01,24.7
2003-01-01,26.8
2004-01-01,29.8
…,…
2019-01-01,267.9
2020-01-01,313.8
2021-01-01,242.2


##### Precio y Variación, Corrientes y Constantes, por NAEH


In [44]:
res_nivel_4(
    "ESR","VAB",
    "Niv_Gr_3",
    "02",
    "Niv_Descr_4")

<string>:29: FutureWarning:

Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.



Niv_Gr_1,Niv_Gr_2,Niv_Gr_3,Niv_Descr_4,N_Variables
str,str,str,str,u32
"""ESR""","""VAB""","""02""",""" Precio """,32
"""ESR""","""VAB""","""02""",""" Variación """,32


In [45]:
res_nivel_5(
    "ESR","VAB",
    "Niv_Gr_3",
    "02",
    "Niv_Descr_4","Niv_Descr_5")

<string>:29: FutureWarning:

Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.



Niv_Gr_1,Niv_Gr_2,Niv_Gr_3,Niv_Descr_4,Niv_Descr_5,N_Variables
str,str,str,str,str,u32
"""ESR""","""VAB""","""02""",""" Precio """,""" Constantes """,16
"""ESR""","""VAB""","""02""",""" Precio """,""" Corrientes """,16
"""ESR""","""VAB""","""02""",""" Variación """,""" Constantes """,16
"""ESR""","""VAB""","""02""",""" Variación """,""" Corrientes """,16


Notar que aquí el Nivel de Descripción 6 (Niv_Descr_6) no es una buena opción de desagregación, puesto que los componentes son "Actividad Económica" y "Productos Alimenticios, Bebidas y Tabaco"; esta última está detallada en mayor grado en el Nivel de Descripción 7 (Niv_Descr_7).

Para saber el contenido de todas las variables:


In [46]:
res_nivel_7(
    "ESR","VAB",
    "Niv_Gr_3",
    "02",
    "Id","Niv_Descr_4","Niv_Descr_5","Niv_Descr_7")

<string>:29: FutureWarning:

Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.



Niv_Gr_1,Niv_Gr_2,Niv_Gr_3,Id,Niv_Descr_4,Niv_Descr_5,Niv_Descr_7,N_Variables
str,str,str,i64,str,str,str,u32
"""ESR""","""VAB""","""02""",297,""" Precio """,""" Corrientes """,""" Muebles""",1
"""ESR""","""VAB""","""02""",310,""" Variación """,""" Constantes """,""" Fabricación d…",1
"""ESR""","""VAB""","""02""",316,""" Variación """,""" Constantes """,""" Producción, P…",1
"""ESR""","""VAB""","""02""",292,""" Precio """,""" Corrientes """,""" Fabricación d…",1
"""ESR""","""VAB""","""02""",301,""" Precio """,""" Corrientes """,""" Elaboración d…",1
…,…,…,…,…,…,…,…
"""ESR""","""VAB""","""02""",315,""" Variación """,""" Constantes """,""" Valor Agregad…",1
"""ESR""","""VAB""","""02""",318,""" Variación """,""" Constantes """,""" Elaboración d…",1
"""ESR""","""VAB""","""02""",327,""" Variación """,""" Corrientes """,""" Industrias Me…",1


Resultados de una sola variable:


In [47]:
Id = 273
df = get_df(Id)
# get_plot(Id)
df

<string>:29: FutureWarning:

Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.



<string>:29: FutureWarning:

Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.



Fecha,"ESR-VAB-02 - Precio - Constantes - Actividad Económica - Productos Alimenticios, Bebidas y Tabaco"
date,f64
2000-01-01,8845.2
2001-01-01,9022.1
2002-01-01,9876.5
2003-01-01,10244.4
2004-01-01,10646.6
…,…
2019-01-01,19358.9
2020-01-01,18674.2
2021-01-01,19897.0


#### PIBA = PIB Anual

La información se encuentra en la viñeta "Sector Real" de la [página web del BCH](https://www.bch.hn/estadisticas-y-publicaciones-economicas/reportes-dinamicos/sector-real).


In [48]:
res_nivel_3("ESR","PIBA","Niv_Descr_3")

4 subgrupos
118 variables


<string>:29: FutureWarning:

Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.



Niv_Gr_1,Niv_Gr_2,Niv_Descr_3,N_Variables
str,str,str,u32
"""ESR""","""PIBA""","""01 """,84
"""ESR""","""PIBA""","""02 """,18
"""ESR""","""PIBA""","""03 """,10
"""ESR""","""PIBA""","""04 """,6


##### 01 Valores Corrientes y Constantes, por Actividad Económica


In [49]:
res_nivel_4(
    "ESR","PIBA",
    "Niv_Gr_3",
    "01",
    "Niv_Descr_4")

<string>:29: FutureWarning:

Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.



Niv_Gr_1,Niv_Gr_2,Niv_Gr_3,Niv_Descr_4,N_Variables
str,str,str,str,u32
"""ESR""","""PIBA""","""01""",""" Constantes """,42
"""ESR""","""PIBA""","""01""",""" Corrientes """,34
"""ESR""","""PIBA""","""01""",""" Corrientes """,8


* Rama de Actividad Económica (RAE)
* Series Agregadas
* Variación RAE
* Variación Series Agregadas


In [50]:
res_nivel_6(
    "ESR","PIBA",
    "Niv_Gr_3",
    "01",
    "Niv_Descr_4","Niv_Descr_5","Niv_Descr_6")

<string>:29: FutureWarning:

Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.



Niv_Gr_1,Niv_Gr_2,Niv_Gr_3,Niv_Descr_4,Niv_Descr_5,Niv_Descr_6,N_Variables
str,str,str,str,str,str,u32
"""ESR""","""PIBA""","""01""",""" Constantes """,""" Rama de Activ…",""" Comunicacione…",1
"""ESR""","""PIBA""","""01""",""" Constantes """,""" Rama de Activ…",""" Electricidad …",1
"""ESR""","""PIBA""","""01""",""" Constantes """,""" Series Agrega…",""" Valor Agregad…",1
"""ESR""","""PIBA""","""01""",""" Constantes """,""" Variación RAE…",""" Electricidad …",1
"""ESR""","""PIBA""","""01""",""" Corrientes """,""" Variación RAE…",""" Intermediació…",1
…,…,…,…,…,…,…
"""ESR""","""PIBA""","""01""",""" Constantes """,""" Rama de Activ…",""" Comercio""",1
"""ESR""","""PIBA""","""01""",""" Constantes """,""" Rama de Activ…",""" Explotación d…",1
"""ESR""","""PIBA""","""01""",""" Constantes """,""" Variación RAE…",""" Servicios Soc…",1


Para saber el contenido de todas las variables:


In [51]:
res_nivel_7(
    "ESR","PIBA",
    "Niv_Gr_3",
    "01",
    "Id","Niv_Descr_4","Niv_Descr_5","Niv_Descr_6")

<string>:29: FutureWarning:

Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.



Niv_Gr_1,Niv_Gr_2,Niv_Gr_3,Id,Niv_Descr_4,Niv_Descr_5,Niv_Descr_6,N_Variables
str,str,str,i64,str,str,str,u32
"""ESR""","""PIBA""","""01""",107,""" Constantes """,""" Rama de Activ…",""" Hoteles y Res…",1
"""ESR""","""PIBA""","""01""",126,""" Constantes """,""" Variación RAE…",""" Electricidad …",1
"""ESR""","""PIBA""","""01""",131,""" Constantes """,""" Variación RAE…",""" Propiedad de …",1
"""ESR""","""PIBA""","""01""",135,""" Constantes """,""" Variación RAE…",""" Servicios Soc…",1
"""ESR""","""PIBA""","""01""",137,""" Constantes """,""" Variacion Ser…",""" Producto Inte…",1
…,…,…,…,…,…,…,…
"""ESR""","""PIBA""","""01""",182,""" Corrientes """,""" Variacion Ser…",""" Valor Agregad…",1
"""ESR""","""PIBA""","""01""",105,""" Constantes """,""" Rama de Activ…",""" Electricidad …",1
"""ESR""","""PIBA""","""01""",141,""" Corrientes """,""" Rama de Activ…",""" Actividades I…",1


Resultados de una sola variable:


In [52]:
Id = 101
df = get_df(Id)
# get_plot(Id)
df

<string>:29: FutureWarning:

Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.



<string>:29: FutureWarning:

Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.



Fecha,"ESR-PIBA-01 - Constantes - Rama de Actividad Económica - Agricultura, Ganadería, Caza, Silvicultura y Pesca"
date,f64
2000-01-01,15328.9
2001-01-01,15654.0
2002-01-01,16358.9
2003-01-01,16743.2
2004-01-01,17952.3
…,…
2019-01-01,30626.7
2020-01-01,28987.9
2021-01-01,28536.8


##### 02 Precios y Variación, por Componente del Gasto


In [53]:
res_nivel_4(
    "ESR","PIBA",
    "Niv_Gr_3",
    "02",
    "Niv_Descr_4")

<string>:29: FutureWarning:

Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.



Niv_Gr_1,Niv_Gr_2,Niv_Gr_3,Niv_Descr_4,N_Variables
str,str,str,str,u32
"""ESR""","""PIBA""","""02""",""" Precio """,10
"""ESR""","""PIBA""","""02""",""" Variación """,8


Tipo de Valores ("Niv_Descr_5"):

* Corrientes
* Constantes


In [54]:
res_nivel_6(
    "ESR","PIBA",
    "Niv_Gr_3",
    "02",
    "Niv_Descr_4","Niv_Descr_5","Niv_Descr_6")

<string>:29: FutureWarning:

Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.



Niv_Gr_1,Niv_Gr_2,Niv_Gr_3,Niv_Descr_4,Niv_Descr_5,Niv_Descr_6,N_Variables
str,str,str,str,str,str,u32
"""ESR""","""PIBA""","""02""",""" Precio """,""" Corrientes """,""" Variación de …",1
"""ESR""","""PIBA""","""02""",""" Precio """,""" Corrientes """,""" Exportación f…",1
"""ESR""","""PIBA""","""02""",""" Variación """,""" Corrientes """,""" Importación c…",1
"""ESR""","""PIBA""","""02""",""" Precio """,""" Constantes """,""" Gasto de Cons…",1
"""ESR""","""PIBA""","""02""",""" Precio """,""" Corrientes """,""" Formación Bru…",1
…,…,…,…,…,…,…
"""ESR""","""PIBA""","""02""",""" Variación """,""" Constantes """,""" Importación c…",1
"""ESR""","""PIBA""","""02""",""" Variación """,""" Corrientes """,""" Formación Bru…",1
"""ESR""","""PIBA""","""02""",""" Precio """,""" Constantes """,""" Exportación f…",1


Para saber el contenido de todas las variables:


In [55]:
res_nivel_7(
    "ESR","PIBA",
    "Niv_Gr_3",
    "02",
    "Id","Niv_Descr_4","Niv_Descr_5","Niv_Descr_6")

<string>:29: FutureWarning:

Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.



Niv_Gr_1,Niv_Gr_2,Niv_Gr_3,Id,Niv_Descr_4,Niv_Descr_5,Niv_Descr_6,N_Variables
str,str,str,i64,str,str,str,u32
"""ESR""","""PIBA""","""02""",184,""" Precio """,""" Constantes """,""" Formación Bru…",1
"""ESR""","""PIBA""","""02""",190,""" Precio """,""" Corrientes """,""" Gasto de Cons…",1
"""ESR""","""PIBA""","""02""",188,""" Precio """,""" Corrientes """,""" Exportación f…",1
"""ESR""","""PIBA""","""02""",191,""" Precio """,""" Corrientes """,""" Importación c…",1
"""ESR""","""PIBA""","""02""",185,""" Precio """,""" Constantes """,""" Gasto de Cons…",1
…,…,…,…,…,…,…,…
"""ESR""","""PIBA""","""02""",192,""" Precio """,""" Corrientes """,""" Variación de …",1
"""ESR""","""PIBA""","""02""",199,""" Variación """,""" Corrientes """,""" Gasto de Cons…",1
"""ESR""","""PIBA""","""02""",183,""" Precio """,""" Constantes """,""" Exportación f…",1


Resultados de una sola variable:


In [56]:
Id = 194
df = get_df(Id)
# get_plot(Id)
df

<string>:29: FutureWarning:

Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.



<string>:29: FutureWarning:

Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.



Fecha,ESR-PIBA-02 - Variación - Constantes - Formación Bruta de Capital Fijo
date,f64
2001-01-01,-3.0
2002-01-01,-7.3
2003-01-01,6.2
2004-01-01,23.4
2005-01-01,-1.4
…,…
2019-01-01,-3.0
2020-01-01,-26.7
2021-01-01,41.5


##### 03 Precios y Variación, por Componente del Ingreso


In [57]:
res_nivel_4(
    "ESR","PIBA",
    "Niv_Gr_3",
    "03",
    "Niv_Descr_4")

<string>:29: FutureWarning:

Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.



Niv_Gr_1,Niv_Gr_2,Niv_Gr_3,Niv_Descr_4,N_Variables
str,str,str,str,u32
"""ESR""","""PIBA""","""03""",""" Variación """,5
"""ESR""","""PIBA""","""03""",""" Precio """,5


* Remuneración de los asalariados
* Impuestos netos de subvenciones sobre la producción y las importaciones
* Excedente de Explotación bruto
* Ingreso Mixto bruto
* Producto Interno Bruto (pm)


In [58]:
res_nivel_5(
    "ESR","PIBA",
    "Niv_Gr_3",
    "03",
    "Niv_Descr_4","Niv_Descr_5")

<string>:29: FutureWarning:

Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.



Niv_Gr_1,Niv_Gr_2,Niv_Gr_3,Niv_Descr_4,Niv_Descr_5,N_Variables
str,str,str,str,str,u32
"""ESR""","""PIBA""","""03""",""" Precio """,""" Impuestos net…",1
"""ESR""","""PIBA""","""03""",""" Precio """,""" Ingreso Mixto…",1
"""ESR""","""PIBA""","""03""",""" Variación """,""" Remuneración …",1
"""ESR""","""PIBA""","""03""",""" Precio """,""" Excedente de …",1
"""ESR""","""PIBA""","""03""",""" Precio """,""" Producto Inte…",1
"""ESR""","""PIBA""","""03""",""" Variación """,""" Producto Inte…",1
"""ESR""","""PIBA""","""03""",""" Variación """,""" Ingreso Mixto…",1
"""ESR""","""PIBA""","""03""",""" Variación """,""" Impuestos net…",1
"""ESR""","""PIBA""","""03""",""" Variación """,""" Excedente de …",1


Para saber el contenido de todas las variables:


In [59]:
res_nivel_7(
    "ESR","PIBA",
    "Niv_Gr_3",
    "03",
    "Id","Nombre","Niv_Descr_4","Niv_Descr_5")

<string>:29: FutureWarning:

Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.



Niv_Gr_1,Niv_Gr_2,Niv_Gr_3,Id,Nombre,Niv_Descr_4,Niv_Descr_5,N_Variables
str,str,str,i64,str,str,str,u32
"""ESR""","""PIBA""","""03""",210,"""ESR-PIBA-03-7""",""" Variación """,""" Remuneración …",1
"""ESR""","""PIBA""","""03""",201,"""ESR-PIBA-03-4""",""" Precio """,""" Excedente de …",1
"""ESR""","""PIBA""","""03""",207,"""ESR-PIBA-03-8""",""" Variación """,""" Impuestos net…",1
"""ESR""","""PIBA""","""03""",204,"""ESR-PIBA-03-6""",""" Precio """,""" Producto Inte…",1
"""ESR""","""PIBA""","""03""",205,"""ESR-PIBA-03-1""",""" Precio """,""" Remuneración …",1
"""ESR""","""PIBA""","""03""",206,"""ESR-PIBA-03-9""",""" Variación """,""" Excedente de …",1
"""ESR""","""PIBA""","""03""",202,"""ESR-PIBA-03-3""",""" Precio """,""" Impuestos net…",1
"""ESR""","""PIBA""","""03""",209,"""ESR-PIBA-03-2""",""" Variación """,""" Producto Inte…",1
"""ESR""","""PIBA""","""03""",203,"""ESR-PIBA-03-5""",""" Precio """,""" Ingreso Mixto…",1


Resultados de una sola variable:


In [60]:
Id = 202
df = get_df(Id)
# get_plot(Id)
df

<string>:29: FutureWarning:

Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.



<string>:29: FutureWarning:

Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.



Fecha,ESR-PIBA-03 - Precio - Impuestos netos de subvenciones sobre la producción y las importaciones
date,f64
2000-01-01,11452.8
2001-01-01,13402.9
2002-01-01,14236.3
2003-01-01,15213.9
2004-01-01,17069.2
…,…
2019-01-01,66960.7
2020-01-01,58433.7
2021-01-01,79447.2


##### 04 Dólares y Lempiras


In [61]:
res_nivel_4(
    "ESR","PIBA",
    "Niv_Gr_3",
    "04",
    "Niv_Descr_4")

<string>:29: FutureWarning:

Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.



Niv_Gr_1,Niv_Gr_2,Niv_Gr_3,Niv_Descr_4,N_Variables
str,str,str,str,u32
"""ESR""","""PIBA""","""04""",""" Lempiras """,3
"""ESR""","""PIBA""","""04""",""" Dólares """,3


* Ingreso Nacional
* Producto Interno Bruto
* Producto Nacional Bruto


In [62]:
res_nivel_5(
    "ESR","PIBA",
    "Niv_Gr_3",
    "04",
    "Niv_Descr_4","Niv_Descr_6")

<string>:29: FutureWarning:

Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.



Niv_Gr_1,Niv_Gr_2,Niv_Gr_3,Niv_Descr_4,Niv_Descr_6,N_Variables
str,str,str,str,str,u32
"""ESR""","""PIBA""","""04""",""" Dólares """,""" Producto Inte…",1
"""ESR""","""PIBA""","""04""",""" Lempiras """,""" Producto Inte…",1
"""ESR""","""PIBA""","""04""",""" Dólares """,""" Producto Naci…",1
"""ESR""","""PIBA""","""04""",""" Lempiras """,""" Producto Naci…",1
"""ESR""","""PIBA""","""04""",""" Lempiras """,""" Ingreso Nacio…",1
"""ESR""","""PIBA""","""04""",""" Dólares """,""" Ingreso Nacio…",1


Para saber el contenido de todas las variables:


In [63]:
res_nivel_7(
    "ESR","PIBA",
    "Niv_Gr_3",
    "04",
    "Id","Nombre","Niv_Descr_4","Niv_Descr_6")

<string>:29: FutureWarning:

Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.



Niv_Gr_1,Niv_Gr_2,Niv_Gr_3,Id,Nombre,Niv_Descr_4,Niv_Descr_6,N_Variables
str,str,str,i64,str,str,str,u32
"""ESR""","""PIBA""","""04""",215,"""ESR-PIBA-04-3""",""" Lempiras """,""" Producto Inte…",1
"""ESR""","""PIBA""","""04""",211,"""ESR-PIBA-04-2""",""" Dólares """,""" Ingreso Nacio…",1
"""ESR""","""PIBA""","""04""",213,"""ESR-PIBA-04-6""",""" Dólares """,""" Producto Naci…",1
"""ESR""","""PIBA""","""04""",212,"""ESR-PIBA-04-4""",""" Dólares """,""" Producto Inte…",1
"""ESR""","""PIBA""","""04""",216,"""ESR-PIBA-04-5""",""" Lempiras """,""" Producto Naci…",1
"""ESR""","""PIBA""","""04""",214,"""ESR-PIBA-04-1""",""" Lempiras """,""" Ingreso Nacio…",1


Resultados de una sola variable:


In [64]:
Id = 211
df = get_df(Id)
# get_plot(Id)
df

<string>:29: FutureWarning:

Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.



<string>:29: FutureWarning:

Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.



Fecha,ESR-PIBA-04 - Dólares - IngresoNacionalDolares - Ingreso Nacional
date,f64
2000-01-01,929.4
2001-01-01,957.5
2002-01-01,958.9
2003-01-01,979.3
2004-01-01,1019.1
…,…
2019-01-01,2098.4
2020-01-01,1961.0
2021-01-01,2261.7


#### PIBT = PIB Trimestral

Esta información se obtiene a partir de los datos del [Portal de Consulta del Sistema de Cuentas Trimestrales](https://sisee.bch.hn/SCNT/PortalPrincipal.aspx).


In [65]:
res_nivel_3("ESR","PIBT","Niv_Gr_3")

3 subgrupos
97 variables


<string>:29: FutureWarning:

Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.



Niv_Gr_1,Niv_Gr_2,Niv_Gr_3,N_Variables
str,str,str,u32
"""ESR""","""PIBT""","""PROD""",68
"""ESR""","""PIBT""","""GAST""",28
"""ESR""","""PIBT""","""TDC""",1


* PROD = PIB Trimestral por el Enfoque de la Producción
* GAST = PIB Trimestral por el Enfoque del Gasto
* TDC = Tasas de Crecimiento

##### PROD = PIB Trimestral por el Enfoque de la Producción

* Serie Original
* Serie Desestacionalizada
* Valores Corrientes
* Valores Constantes

Para saber el contenido de todas las variables:


In [66]:
res_nivel_7(
    "ESR","PIBT",
    "Niv_Gr_3",
    "PROD",
    "Id","Niv_Descr_1","Niv_Descr_2","Niv_Descr_4")

<string>:29: FutureWarning:

Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.



Niv_Gr_1,Niv_Gr_2,Niv_Gr_3,Id,Niv_Descr_1,Niv_Descr_2,Niv_Descr_4,N_Variables
str,str,str,i64,str,str,str,u32
"""ESR""","""PIBT""","""PROD""",22,"""Serie Desestac…","""VALORES CORRIE…","""Administración…",1
"""ESR""","""PIBT""","""PROD""",51,"""Serie Original…","""VALORES CORRIE…","""Electricidad y…",1
"""ESR""","""PIBT""","""PROD""",89,"""Serie Original…","""VALORES CORRIE…","""Valor Agregado…",1
"""ESR""","""PIBT""","""PROD""",16,"""Serie Desestac…","""VALORES CORRIE…","""Comercio Repar…",1
"""ESR""","""PIBT""","""PROD""",47,"""Serie Desestac…","""VALORES CORRIE…","""Propiedad de V…",1
…,…,…,…,…,…,…,…
"""ESR""","""PIBT""","""PROD""",19,"""Serie Desestac…","""VALORES CORRIE…","""Industrias Man…",1
"""ESR""","""PIBT""","""PROD""",96,"""Serie Original…","""VALORES CORRIE…","""Intermediación…",1
"""ESR""","""PIBT""","""PROD""",15,"""Serie Desestac…","""VALORES CORRIE…","""Transporte y A…",1


Resultados de una sola variable:


In [67]:
Id = 2
df = get_df(Id)
# get_plot(Id)
df

<string>:29: FutureWarning:

Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.



<string>:29: FutureWarning:

Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.



Fecha,Serie Desestacionalizada-VALORES CORRIENTES-14-Servicios de Enseñanza Servicios Sociales y de Salud
date,f64
2000-03-01,1754.8
2000-06-01,1871.8
2000-09-01,1939.6
2000-12-01,1938.9
2001-03-01,2250.4
…,…
2022-12-01,18497.7
2023-03-01,18732.8
2023-06-01,19374.7


##### GAST = PIB Trimestral por el Enfoque del Gasto

* Serie Original
* Serie Desestacionalizada
* Valores Corrientes
* Valores Constantes

Para saber el contenido de todas las variables:


In [68]:
res_nivel_7(
    "ESR","PIBT",
    "Niv_Gr_3",
    "GAST",
    "Id","Niv_Descr_1","Niv_Descr_2","Niv_Descr_4")

<string>:29: FutureWarning:

Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.



Niv_Gr_1,Niv_Gr_2,Niv_Gr_3,Id,Niv_Descr_1,Niv_Descr_2,Niv_Descr_4,N_Variables
str,str,str,i64,str,str,str,u32
"""ESR""","""PIBT""","""GAST""",10,"""Serie Original…","""VALORES CONSTA…","""Sector Público…",1
"""ESR""","""PIBT""","""GAST""",25,"""Serie Desestac…","""VALORES CORRIE…","""Producto Inter…",1
"""ESR""","""PIBT""","""GAST""",31,"""Serie Desestac…","""VALORES CORRIE…","""Sector Público…",1
"""ESR""","""PIBT""","""GAST""",35,"""Serie Desestac…","""VALORES CONSTA…","""Formación Brut…",1
"""ESR""","""PIBT""","""GAST""",42,"""Serie Original…","""VALORES CORRIE…","""Importación de…",1
…,…,…,…,…,…,…,…
"""ESR""","""PIBT""","""GAST""",12,"""Serie Original…","""VALORES CONSTA…","""Formación Brut…",1
"""ESR""","""PIBT""","""GAST""",37,"""Serie Original…","""VALORES CORRIE…","""Gastos de Cons…",1
"""ESR""","""PIBT""","""GAST""",26,"""Serie Desestac…","""VALORES CONSTA…","""Exportaciones …",1


Resultados de una sola variable:


In [69]:
Id = 8
df = get_df(Id)
# get_plot(Id)
df

<string>:29: FutureWarning:

Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.



<string>:29: FutureWarning:

Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.



Fecha,Serie Original-VALORES CONSTANTES-1-Gastos de Consumo Final
date,f64
2000-03-01,21983.9
2000-06-01,22187.9
2000-09-01,22183.0
2000-12-01,23404.6
2001-03-01,22815.4
…,…
2022-12-01,60656.4
2023-03-01,55847.1
2023-06-01,58735.5


##### TDC = Tasas de Crecimiento

* Serie Original
* Serie Desestacionalizada
* Valores Corrientes
* Valores Constantes

Para saber el contenido de todas las variables:


In [70]:
res_nivel_6(
    "ESR","PIBT",
    "Niv_Gr_3",
    "TDC",
    "Id","Nombre","Niv_Descr_1")

<string>:29: FutureWarning:

Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.



Niv_Gr_1,Niv_Gr_2,Niv_Gr_3,Id,Nombre,Niv_Descr_1,N_Variables
str,str,str,i64,str,str,u32
"""ESR""","""PIBT""","""TDC""",98,"""ESR-PIBT-TDC-V…","""Valores Ajusta…",1


Resultados de una sola variable:


In [71]:
Id = 98
df = get_df(Id)
# get_plot(Id)
df

<string>:29: FutureWarning:

Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.



<string>:29: FutureWarning:

Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.



Fecha,Valores Ajustados
date,f64
2000-03-01,26393.4
2000-06-01,26528.2
2000-09-01,26710.9
2000-12-01,26992.4
2001-03-01,27225.7
…,…
2022-12-01,59862.9
2023-03-01,59727.1
2023-06-01,60564.6


#### ODA = Oferta y Demanda Agregada

* Oferta Global
* Demanda Global


In [72]:
res_nivel_3("ESR","ODA","Niv_Descr_1")

2 subgrupos
94 variables


<string>:29: FutureWarning:

Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.



Niv_Gr_1,Niv_Gr_2,Niv_Descr_1,N_Variables
str,str,str,u32
"""ESR""","""ODA""","""OFERTA GLOBAL""",48
"""ESR""","""ODA""","""DEMANDA GLOBAL…",46


##### Oferta Global

Estas tablas están agrupadas de dos maneras:

1. Variables con valores en "Niv_Descr_2" y "Niv_Descr_3", corresponde a las siguientes series:

* AGRICULTURA, GANADERÍA, CAZA, SILVICULTURA Y PESCA (A PRECIOS BÁSICOS)
* COMERCIO, HOTELES Y RESTAURANTES (A PRECIOS BÁSICOS)
* CONSTRUCCIÓN (A PRECIOS BÁSICOS)
* EXPLOTACIÓN DE MINAS Y CANTERAS (A PRECIOS BÁSICOS)
* INDUSTRIA MANUFACTURERA (A PRECIOS BÁSICOS)
* MENOS: SERVICIOS DE INTERMEDIACIÓN FINANCIERA MEDIDOS INDIRECTAMENTE (A PRECIOS BÁSICOS)

El tipo de valor ("Niv_Descr_4") de dichas variables es:

* EN MILLONES DE LEMPIRAS CONSTANTES
* EN MILLONES DE LEMPIRAS CORRIENTES
* VARIACIONES RELATIVAS CONSTANTE
* VARIACIONES RELATIVAS CORRIENTES


In [73]:
res_nivel_7(
    "ESR","ODA",
    "Niv_Descr_1",
    "OFERTA GLOBAL",
    "Id","Niv_Descr_2","Niv_Descr_3","Niv_Descr_4"
    ).filter((pl.col('Niv_Descr_2') != ""))

<string>:29: FutureWarning:

Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.



Niv_Gr_1,Niv_Gr_2,Niv_Descr_1,Id,Niv_Descr_2,Niv_Descr_3,Niv_Descr_4,N_Variables
str,str,str,i64,str,str,str,u32
"""ESR""","""ODA""","""OFERTA GLOBAL""",11498,"""VALOR AGREGADO…","""AGRICULTURA, G…","""VARIACIONES RE…",1
"""ESR""","""ODA""","""OFERTA GLOBAL""",11503,"""VALOR AGREGADO…","""COMERCIO, HOTE…","""VARIACIONES RE…",1
"""ESR""","""ODA""","""OFERTA GLOBAL""",11504,"""VALOR AGREGADO…","""CONSTRUCCIÓN (…","""EN MILLONES DE…",1
"""ESR""","""ODA""","""OFERTA GLOBAL""",11506,"""VALOR AGREGADO…","""CONSTRUCCIÓN (…","""VARIACIONES RE…",1
"""ESR""","""ODA""","""OFERTA GLOBAL""",11509,"""VALOR AGREGADO…","""EXPLOTACIÓN DE…","""EN MILLONES DE…",1
…,…,…,…,…,…,…,…
"""ESR""","""ODA""","""OFERTA GLOBAL""",11501,"""VALOR AGREGADO…","""COMERCIO, HOTE…","""EN MILLONES DE…",1
"""ESR""","""ODA""","""OFERTA GLOBAL""",11502,"""VALOR AGREGADO…","""COMERCIO, HOTE…","""VARIACIONES RE…",1
"""ESR""","""ODA""","""OFERTA GLOBAL""",11517,"""VALOR AGREGADO…","""MENOS: SERVICI…","""EN MILLONES DE…",1


Resultados de una sola variable:


In [74]:
Id = 11518
df = get_df(Id)
# get_plot(Id)
df

<string>:29: FutureWarning:

Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.



<string>:29: FutureWarning:

Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.



Fecha,OFERTA GLOBAL-VALOR AGREGADO BRUTO-MENOS: SERVICIOS DE INTERMEDIACIÓN FINANCIERA MEDIDOS INDIRECTAMENTE (A PRECIOS BÁSICOS)-VARIACIONES RELATIVAS CONSTANTE
date,f64
2001-01-01,10.7
2002-01-01,19.4
2003-01-01,13.3
2004-01-01,16.6
2005-01-01,22.5
…,…
2019-01-01,7.0
2020-01-01,1.8
2021-01-01,13.4


2. Variables sin valores en "Niv_Descr_2" y "Niv_Descr_3", corresponde a las siguientes series:

* IMPORTACIONES DE BIENES Y SERVICIOS C.I.F.
* IMPUESTOS NETOS DE SUBVENCIONES SOBRE LA PRODUCCIÓN Y LAS IMPORTACIONES
* OFERTA GLOBAL
* OTROS SERVICIOS (A PRECIOS BÃSICOS)
* PRODUCTO INTERNO BRUTO A PRECIOS DE MERCADO
* TOTAL VALOR AGREGADO BRUTO A PRECIOS BÁSICOS
* VALOR AGREGADO BRUTO
* VALOR AGREGADO BRUTO A PRECIOS BÁSICOS

El tipo de valor ("Niv_Descr_5") de dichas variables es:

* EN MILLONES DE LEMPIRAS CONSTANTES
* EN MILLONES DE LEMPIRAS CORRIENTES
* VARIACIONES RELATIVAS CONSTANTE
* VARIACIONES RELATIVAS CORRIENTES

Para saber el contenido de todas las variables:


In [75]:
res_nivel_7(
    "ESR","ODA",
    "Niv_Descr_1",
    "OFERTA GLOBAL",
    "Id","Niv_Descr_2","Niv_Descr_4","Niv_Descr_5"
    ).filter((pl.col('Niv_Descr_2') == ""))

<string>:29: FutureWarning:

Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.



Niv_Gr_1,Niv_Gr_2,Niv_Descr_1,Id,Niv_Descr_2,Niv_Descr_4,Niv_Descr_5,N_Variables
str,str,str,i64,str,str,str,u32
"""ESR""","""ODA""","""OFERTA GLOBAL""",11486,"""""","""OTROS SERVICIO…","""VARIACIONES RE…",1
"""ESR""","""ODA""","""OFERTA GLOBAL""",11494,"""""","""VALOR AGREGADO…","""VARIACIONES RE…",1
"""ESR""","""ODA""","""OFERTA GLOBAL""",11472,"""""","""IMPORTACIONES …","""EN MILLONES DE…",1
"""ESR""","""ODA""","""OFERTA GLOBAL""",11483,"""""","""OFERTA GLOBAL""","""VARIACIONES RE…",1
"""ESR""","""ODA""","""OFERTA GLOBAL""",11491,"""""","""PRODUCTO INTER…","""VARIACIONES RE…",1
…,…,…,…,…,…,…,…
"""ESR""","""ODA""","""OFERTA GLOBAL""",11476,"""""","""IMPUESTOS NETO…","""EN MILLONES DE…",1
"""ESR""","""ODA""","""OFERTA GLOBAL""",11481,"""""","""OFERTA GLOBAL""","""EN MILLONES DE…",1
"""ESR""","""ODA""","""OFERTA GLOBAL""",11475,"""""","""IMPORTACIONES …","""VARIACIONES RE…",1


Resultados de una sola variable:


In [76]:
Id = 11473
df = get_df(Id)
# get_plot(Id)
df

<string>:29: FutureWarning:

Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.



<string>:29: FutureWarning:

Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.



Fecha,OFERTA GLOBAL---IMPORTACIONES DE BIENES Y SERVICIOS C.I.F.-EN MILLONES DE LEMPIRAS CORRIENTES
date,f64
2000-01-01,70839.7
2001-01-01,76450.8
2002-01-01,84303.0
2003-01-01,97336.4
2004-01-01,124423.4
…,…
2019-01-01,361267.7
2020-01-01,299223.4
2021-01-01,432645.9


##### Demanda Global

Estas tablas están agrupadas de dos maneras:

1. Variables con valores en "Niv_Descr_2" y "Niv_Descr_3", corresponde a las siguientes series:

* FORMACIÓN BRUTA DE CAPITAL FIJO
* GASTOS DE CONSUMO FINAL

De "Niv_Descr_3" se obtiene:

* SECTOR PRIVADO
* SECTOR PÚBLICO

El tipo de valor ("Niv_Descr_4") de dichas variables es:

* EN MILLONES DE LEMPIRAS CONSTANTES
* EN MILLONES DE LEMPIRAS CORRIENTES
* VARIACIONES RELATIVAS CONSTANTE
* VARIACIONES RELATIVAS CORRIENTES


In [77]:
res_nivel_7(
    "ESR","ODA",
    "Niv_Descr_1",
    "DEMANDA GLOBAL",
    "Id","Niv_Descr_2","Niv_Descr_3","Niv_Descr_4"
    ).filter((pl.col('Niv_Descr_2') != ""))

<string>:29: FutureWarning:

Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.



Niv_Gr_1,Niv_Gr_2,Niv_Descr_1,Id,Niv_Descr_2,Niv_Descr_3,Niv_Descr_4,N_Variables
str,str,str,i64,str,str,str,u32
"""ESR""","""ODA""","""DEMANDA GLOBAL…",11457,"""FORMACIÓN BRUT…","""SECTOR PRIVADO…","""EN MILLONES DE…",1
"""ESR""","""ODA""","""DEMANDA GLOBAL…",11463,"""FORMACIÓN BRUT…","""SECTOR PÚBLICO…","""VARIACIONES RE…",1
"""ESR""","""ODA""","""DEMANDA GLOBAL…",11465,"""GASTOS DE CONS…","""SECTOR PRIVADO…","""EN MILLONES DE…",1
"""ESR""","""ODA""","""DEMANDA GLOBAL…",11464,"""GASTOS DE CONS…","""SECTOR PRIVADO…","""EN MILLONES DE…",1
"""ESR""","""ODA""","""DEMANDA GLOBAL…",11462,"""FORMACIÓN BRUT…","""SECTOR PÚBLICO…","""VARIACIONES RE…",1
…,…,…,…,…,…,…,…
"""ESR""","""ODA""","""DEMANDA GLOBAL…",11461,"""FORMACIÓN BRUT…","""SECTOR PÚBLICO…","""EN MILLONES DE…",1
"""ESR""","""ODA""","""DEMANDA GLOBAL…",11470,"""GASTOS DE CONS…","""SECTOR PÚBLICO…","""VARIACIONES RE…",1
"""ESR""","""ODA""","""DEMANDA GLOBAL…",11467,"""GASTOS DE CONS…","""SECTOR PRIVADO…","""VARIACIONES RE…",1


Resultados de una sola variable:


In [78]:
Id = 11457
df = get_df(Id)
# get_plot(Id)
df

<string>:29: FutureWarning:

Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.



<string>:29: FutureWarning:

Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.



Fecha,DEMANDA GLOBAL-FORMACIÓN BRUTA DE CAPITAL FIJO-SECTOR PRIVADO-EN MILLONES DE LEMPIRAS CORRIENTES
date,f64
2000-01-01,21630.4
2001-01-01,21859.9
2002-01-01,23425.3
2003-01-01,25758.4
2004-01-01,34118.0
…,…
2019-01-01,121220.5
2020-01-01,89277.4
2021-01-01,132749.6


2. Variables sin valores en "Niv_Descr_2" y "Niv_Descr_3", corresponde a las siguientes series:

* DEMANDA EXTERNA
* DEMANDA GLOBAL
* DEMANDA INTERNA
* EXPORTACIÓN DE BIENES Y SERVICIOS F.O.B.
* FORMACIÓN BRUTA DE CAPITAL FIJO
* GASTOS DE CONSUMO FINAL
* INVERSIÓN INTERNA BRUTA
* VARIACIÓN DE EXISTENCIAS

El tipo de valor de dichas variables es:

* EN MILLONES DE LEMPIRAS CONSTANTES
* EN MILLONES DE LEMPIRAS CORRIENTES
* VARIACIONES RELATIVAS CONSTANTE
* VARIACIONES RELATIVAS CORRIENTES

Para saber el contenido de todas las variables:


In [79]:
res_nivel_7(
    "ESR","ODA",
    "Niv_Descr_1",
    "DEMANDA GLOBAL",
    "Id","Niv_Descr_2","Niv_Descr_4","Niv_Descr_5"
    ).filter((pl.col('Niv_Descr_2') == ""))

<string>:29: FutureWarning:

Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.



Niv_Gr_1,Niv_Gr_2,Niv_Descr_1,Id,Niv_Descr_2,Niv_Descr_4,Niv_Descr_5,N_Variables
str,str,str,i64,str,str,str,u32
"""ESR""","""ODA""","""DEMANDA GLOBAL…",11426,"""""","""DEMANDA EXTERN…","""EN MILLONES DE…",1
"""ESR""","""ODA""","""DEMANDA GLOBAL…",11455,"""""","""VARIACIÓN DE E…","""EN MILLONES DE…",1
"""ESR""","""ODA""","""DEMANDA GLOBAL…",11428,"""""","""DEMANDA EXTERN…","""VARIACIONES RE…",1
"""ESR""","""ODA""","""DEMANDA GLOBAL…",11433,"""""","""DEMANDA GLOBAL…","""VARIACIONES RE…",1
"""ESR""","""ODA""","""DEMANDA GLOBAL…",11454,"""""","""VARIACIÓN DE E…","""EN MILLONES DE…",1
…,…,…,…,…,…,…,…
"""ESR""","""ODA""","""DEMANDA GLOBAL…",11444,"""""","""FORMACIÓN BRUT…","""VARIACIONES RE…",1
"""ESR""","""ODA""","""DEMANDA GLOBAL…",11438,"""""","""EXPORTACIÓN DE…","""EN MILLONES DE…",1
"""ESR""","""ODA""","""DEMANDA GLOBAL…",11430,"""""","""DEMANDA GLOBAL…","""EN MILLONES DE…",1


Resultados de una sola variable:


In [80]:
Id = 11440
df = get_df(Id)
# get_plot(Id)
df

<string>:29: FutureWarning:

Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.



<string>:29: FutureWarning:

Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.



Fecha,DEMANDA GLOBAL---EXPORTACIÓN DE BIENES Y SERVICIOS F.O.B.-VARIACIONES RELATIVAS CONSTANTE
date,f64
2001-01-01,4.0
2002-01-01,10.0
2003-01-01,8.7
2004-01-01,13.2
2005-01-01,5.3
…,…
2019-01-01,1.0
2020-01-01,-21.0
2021-01-01,23.2


#### IMAE = Índice Mensual de la Actividad Económica

Las actividades ("Niv_Descr_4") son:

* Agricultura, Ganaderia, Silvicultura y Pesca 
* Minas y Canteras 
*  Industria Manufacturera 
* Electricidad y Agua 
* ConstrucciÃ³n 
* Comercio 
* Hoteles y Restaurantes 
* Transporte y Almacenamiento 
* Correo y Telecomunicaciones 
* Inter. Financiera, Seguros y Fondos de Pensiones 
* Otros Servicios 
* Serie Original 
* Tendencia Ciclo 

El tipo de valor ("Niv_Descr_5") de dichas variables es:

* Acumulada
* Interanual
* Índice


In [81]:
res_nivel_3("ESR","IMAE","Niv_Descr_2")

1 subgrupos
39 variables


<string>:29: FutureWarning:

Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.



Niv_Gr_1,Niv_Gr_2,Niv_Descr_2,N_Variables
str,str,str,u32
"""ESR""","""IMAE""","""IMAE""",39


Para saber el contenido de todas las variables:


In [82]:
res_nivel_6(
    "ESR","IMAE",
    "Niv_Descr_2",
    "IMAE",
    "Id","Niv_Descr_4","Niv_Descr_5")

<string>:29: FutureWarning:

Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.



Niv_Gr_1,Niv_Gr_2,Niv_Descr_2,Id,Niv_Descr_4,Niv_Descr_5,N_Variables
str,str,str,i64,str,str,u32
"""ESR""","""IMAE""","""IMAE""",357,""" Hoteles y Res…",""" Índice""",1
"""ESR""","""IMAE""","""IMAE""",366,""" Inter. Financ…",""" Índice""",1
"""ESR""","""IMAE""","""IMAE""",347,""" Electricidad …",""" Interanual""",1
"""ESR""","""IMAE""","""IMAE""",353,""" Comercio """,""" Interanual""",1
"""ESR""","""IMAE""","""IMAE""",360,""" Transporte y …",""" Índice""",1
…,…,…,…,…,…,…
"""ESR""","""IMAE""","""IMAE""",373,""" Tendencia Cic…",""" Acumulada""",1
"""ESR""","""IMAE""","""IMAE""",374,""" Tendencia Cic…",""" Interanual""",1
"""ESR""","""IMAE""","""IMAE""",341,""" Minas y Cante…",""" Interanual""",1


Resultados de una sola variable:


In [83]:
Id = 372
df = get_df(Id)
# get_plot(Id)
df

<string>:29: FutureWarning:

Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.



<string>:29: FutureWarning:

Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.



Fecha,ESR-IMAE-01 - Serie Original - Índice
date,f64
2000-01-01,96.0836
2000-02-01,100.354
2000-03-01,103.7134
2000-04-01,91.3566
2000-05-01,98.5831
…,…
2023-10-01,282.5159
2023-11-01,265.7474
2023-12-01,304.4677


### ESE = Estadísticas del Sector Externo


In [84]:
res_nivel_2("ESE")

11519 variables
54 grupos
12 variables
6 grupos
12 subgrupos


<string>:29: FutureWarning:

Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.



Niv_Gr_1,Niv_Gr_2,N_Variables,N_Niv_Gr_2
str,str,u32,u32
"""ESE""","""CEB""",4,4
"""ESE""","""DE""",4,4
"""ESE""","""PII""",1,1
"""ESE""","""IED""",1,1
"""ESE""","""IES""",1,1
"""ESE""","""BP""",1,1


* DE = Deuda Externa
* CEB = Comercio Exterior de Bienes
* BP = Balanza de Pagos
* IES = Ingresos y Egresos de Servicios
* PII = Posición de Inversión Inernacional
* IED = Inversión Extranjera Directa

#### DE = Deuda Externa


In [85]:
res_nivel_3("ESE","DE","Niv_Gr_3")

4 subgrupos
737 variables


<string>:29: FutureWarning:

Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.



Niv_Gr_1,Niv_Gr_2,Niv_Gr_3,N_Variables
str,str,str,u32
"""ESE""","""DE""","""03""",672
"""ESE""","""DE""","""04""",55
"""ESE""","""DE""","""02""",6
"""ESE""","""DE""","""01""",4


##### 01 - Deudor

Para ver todas las variables:


In [86]:
res_nivel_7(
    "ESE","DE",
    "Niv_Gr_3","01",
    "Id","Nombre","Niv_Descr_1","Niv_Descr_2").sort("Nombre")

<string>:29: FutureWarning:

Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.



Niv_Gr_1,Niv_Gr_2,Niv_Gr_3,Id,Nombre,Niv_Descr_1,Niv_Descr_2,N_Variables
str,str,str,i64,str,str,str,u32
"""ESE""","""DE""","""01""",6710,"""ESE-DE-01-1""","""Deudor""","""Autoridad Mone…",1
"""ESE""","""DE""","""01""",6711,"""ESE-DE-01-2""","""Deudor""","""Empresas Públi…",1
"""ESE""","""DE""","""01""",6712,"""ESE-DE-01-3""","""Deudor""","""Gobierno Gener…",1
"""ESE""","""DE""","""01""",6713,"""ESE-DE-01-4""","""Deudor""","""Instituciones …",1


Resultados de una sola variable:


In [87]:
Id = 6710
df = get_df(Id)
# get_plot(Id)
df

<string>:29: FutureWarning:

Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.



<string>:29: FutureWarning:

Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.



Fecha,Deudor-Autoridad Monetaria
date,f64
2008-01-31,172.8
2008-02-29,173.2
2008-03-31,173.8
2008-04-30,173.4
2008-05-31,171.9
…,…
2023-12-30,648.4
2023-12-31,648.7
2024-01-31,643.3


##### 02 - Acreedor

Para ver todas las variables:


In [88]:
res_nivel_7(
    "ESE","DE",
    "Niv_Gr_3","02",
    "Id","Nombre","Niv_Descr_1","Niv_Descr_2").sort("Nombre")

<string>:29: FutureWarning:

Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.



Niv_Gr_1,Niv_Gr_2,Niv_Gr_3,Id,Nombre,Niv_Descr_1,Niv_Descr_2,N_Variables
str,str,str,i64,str,str,str,u32
"""ESE""","""DE""","""02""",6714,"""ESE-DE-02-1""","""Acreedor""","""Sector Privado…",1
"""ESE""","""DE""","""02""",6715,"""ESE-DE-02-2""","""Acreedor""","""Sector Público…",1
"""ESE""","""DE""","""02""",6716,"""ESE-DE-02-3""","""Acreedor""","""Sector Público…",1
"""ESE""","""DE""","""02""",6717,"""ESE-DE-02-4""","""Acreedor""","""Sector Privado…",1
"""ESE""","""DE""","""02""",6718,"""ESE-DE-02-5""","""Acreedor""","""Sector Público…",1
"""ESE""","""DE""","""02""",6719,"""ESE-DE-02-6""","""Acreedor""","""Sector Privado…",1


Resultados de una sola variable:


In [89]:
Id = 6714
df = get_df(Id)
# get_plot(Id)
df

<string>:29: FutureWarning:

Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.



<string>:29: FutureWarning:

Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.



Fecha,Acreedor-Sector Privado Bilateral
date,f64
2008-03-31,30.7
2008-06-30,28.8
2008-09-30,23.1
2008-12-31,30.7
2009-03-31,34.3
…,…
2023-03-31,162.6
2023-06-30,144.7
2023-09-30,153.4


##### 03 - Por Institución

* Banco Central
* Gobierno general
* Otras sociedades financieras
* Otros Sectores
* Saldo bruto de la deuda externa
* Sociedades no financieras


In [90]:
res_nivel_4(
    "ESE","DE",
    "Niv_Gr_3",
    "03",
    "Niv_Descr_1")

<string>:29: FutureWarning:

Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.



Niv_Gr_1,Niv_Gr_2,Niv_Gr_3,Niv_Descr_1,N_Variables
str,str,str,str,u32
"""ESE""","""DE""","""03""","""Gobierno gener…",144
"""ESE""","""DE""","""03""","""Banco Central""",144
"""ESE""","""DE""","""03""","""Otros Sectores…",120
"""ESE""","""DE""","""03""","""Otras sociedad…",120
"""ESE""","""DE""","""03""","""Sociedades no …",120
"""ESE""","""DE""","""03""","""Saldo bruto de…",24


* Asignación de Derechos especiales de giro
* Asignación de Derechos especiales de giro 
* Créditos y anticipos comerciales
* Créditos y anticipos comerciales 
* Moneda y depósitos
* Moneda y depósitos 
* Otros pasivos de deuda
* Otros pasivos de deuda 
* Préstamos
* Préstamos 
* Tí­tulos de deuda
* Tí­tulos de deuda 
* Saldo bruto de la deuda externa
* Saldo bruto de la deuda externa 


In [91]:
res_nivel_4(
    "ESE","DE",
    "Niv_Gr_3",
    "03",
    "Niv_Descr_2")

<string>:29: FutureWarning:

Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.



Niv_Gr_1,Niv_Gr_2,Niv_Gr_3,Niv_Descr_2,N_Variables
str,str,str,str,u32
"""ESE""","""DE""","""03""","""Créditos y ant…",80
"""ESE""","""DE""","""03""","""Préstamos """,80
"""ESE""","""DE""","""03""","""Otros pasivos …",80
"""ESE""","""DE""","""03""","""Moneda y depós…",80
"""ESE""","""DE""","""03""","""Títulos de deu…",80
…,…,…,…,…
"""ESE""","""DE""","""03""","""Moneda y depós…",40
"""ESE""","""DE""","""03""","""Asignación de …",32
"""ESE""","""DE""","""03""","""Saldo bruto de…",16


In [92]:
res_nivel_7(
    "ESE","DE",
    "Niv_Gr_3","03",
    "Id","Niv_Descr_1","Niv_Descr_2","Niv_Descr_3")

<string>:29: FutureWarning:

Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.



Niv_Gr_1,Niv_Gr_2,Niv_Gr_3,Id,Niv_Descr_1,Niv_Descr_2,Niv_Descr_3,N_Variables
str,str,str,i64,str,str,str,u32
"""ESE""","""DE""","""03""",6722,"""Banco Central""","""Asignación de …","""18""",1
"""ESE""","""DE""","""03""",6764,"""Banco Central""","""Créditos y ant…",""" Principal""",1
"""ESE""","""DE""","""03""",6785,"""Banco Central""","""Moneda y depós…",""" Principal""",1
"""ESE""","""DE""","""03""",6790,"""Banco Central""","""Moneda y depós…",""" Principal""",1
"""ESE""","""DE""","""03""",6798,"""Banco Central""","""Otros pasivos …","""9""",1
…,…,…,…,…,…,…,…
"""ESE""","""DE""","""03""",7342,"""Sociedades no …","""Otros pasivos …",""" Principal""",1
"""ESE""","""DE""","""03""",7343,"""Sociedades no …","""Otros pasivos …",""" Principal""",1
"""ESE""","""DE""","""03""",7355,"""Sociedades no …","""Préstamos """,""" Intereses""",1


Para ver todas las variables (Nota: el nivel de descripción sigue):


In [93]:
df_all.filter(
    pl.col("Niv_Gr_1") == "ESE",
    pl.col("Niv_Gr_2") == "DE",
    pl.col("Niv_Gr_3") == "03"
    ).select(pl.all().exclude(
        "Niv_Descr_6","Niv_Descr_7","Niv_Descr_8",
        "Niv_Gr_4","Niv_Gr_5"))

Id,Nombre,Descripcion,Periodicidad,Grupo,CorrelativoGrupo,Niv_Descr_1,Niv_Descr_2,Niv_Descr_3,Niv_Descr_4,Niv_Descr_5,Niv_Descr,Niv_Gr_1,Niv_Gr_2,Niv_Gr_3,Niv_Gr
i64,str,str,str,str,str,str,str,str,str,str,i64,str,str,str,i64
6720,"""ESE-DE-03-1""","""Banco Central-…","""Trimestral""","""ESE-DE-03""","""1""","""Banco Central""","""Asignación de …","""0""","""3 meses""",null,4,"""ESE""","""DE""","""03""",12
6721,"""ESE-DE-03-2""","""Banco Central-…","""Trimestral""","""ESE-DE-03""","""2""","""Banco Central""","""Asignación de …","""12""","""18 meses""",null,4,"""ESE""","""DE""","""03""",12
6722,"""ESE-DE-03-3""","""Banco Central-…","""Trimestral""","""ESE-DE-03""","""3""","""Banco Central""","""Asignación de …","""18""","""24 meses""",null,4,"""ESE""","""DE""","""03""",12
6723,"""ESE-DE-03-4""","""Banco Central-…","""Trimestral""","""ESE-DE-03""","""4""","""Banco Central""","""Asignación de …","""2 años en adel…",null,null,3,"""ESE""","""DE""","""03""",11
6724,"""ESE-DE-03-5""","""Banco Central-…","""Trimestral""","""ESE-DE-03""","""5""","""Banco Central""","""Asignación de …","""3""","""6 meses""",null,4,"""ESE""","""DE""","""03""",12
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
7387,"""ESE-DE-03-668""","""Sociedades no …","""Trimestral""","""ESE-DE-03""","""668""","""Sociedades no …","""Títulos de deu…",""" Principal""","""2 años en adel…",null,4,"""ESE""","""DE""","""03""",12
7388,"""ESE-DE-03-669""","""Sociedades no …","""Trimestral""","""ESE-DE-03""","""669""","""Sociedades no …","""Títulos de deu…",""" Principal""","""3""","""6 meses""",5,"""ESE""","""DE""","""03""",13
7389,"""ESE-DE-03-670""","""Sociedades no …","""Trimestral""","""ESE-DE-03""","""670""","""Sociedades no …","""Títulos de deu…",""" Principal""","""6""","""9 meses""",5,"""ESE""","""DE""","""03""",13


Resultados de una sola variable:


In [94]:
Id = 6720
df = get_df(Id)
# get_plot(Id)
df

<string>:29: FutureWarning:

Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.



<string>:29: FutureWarning:

Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.



Fecha,Banco Central-Asignación de Derechos especiales de giro-0-3 meses
date,i64
2024-04-19,0


##### 04 - Pagos que vencen en un año o menos

* Gobierno general
* Banco Central
* Otros Sectores
* Otras sociedades financieras
* Sociedades no financieras
* Deuda externa total a corto plazo


In [95]:
res_nivel_4(
    "ESE","DE",
    "Niv_Gr_3",
    "04",
    "Niv_Descr_1")

<string>:29: FutureWarning:

Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.



Niv_Gr_1,Niv_Gr_2,Niv_Gr_3,Niv_Descr_1,N_Variables
str,str,str,str,u32
"""ESE""","""DE""","""04""","""Gobierno gener…",12
"""ESE""","""DE""","""04""","""Banco Central""",12
"""ESE""","""DE""","""04""","""Otras sociedad…",10
"""ESE""","""DE""","""04""","""Otros Sectores…",10
"""ESE""","""DE""","""04""","""Sociedades no …",10
"""ESE""","""DE""","""04""","""Deuda externa …",1


* A corto plazo según el vencimiento original
* A largo plazo exigible dentro de un año o antes
* Deuda externa total a corto plazo


In [96]:
res_nivel_4(
    "ESE","DE",
    "Niv_Gr_3",
    "04",
    "Niv_Descr_2")

<string>:29: FutureWarning:

Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.



Niv_Gr_1,Niv_Gr_2,Niv_Gr_3,Niv_Descr_2,N_Variables
str,str,str,str,u32
"""ESE""","""DE""","""04""","""A corto plazo …",27
"""ESE""","""DE""","""04""","""A largo plazo …",27
"""ESE""","""DE""","""04""","""Deuda externa …",1


* Asignación de Derechos especiales de giro
* Moneda y depósitos
* Tí­tulos de deuda
* Préstamos
* Créditos y anticipos comerciales
* Otros pasivos de deuda
* Deuda externa total a corto plazo (según el vencimiento residual)


In [97]:
res_nivel_4(
    "ESE","DE",
    "Niv_Gr_3",
    "04",
    "Niv_Descr_3")

<string>:29: FutureWarning:

Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.



Niv_Gr_1,Niv_Gr_2,Niv_Gr_3,Niv_Descr_3,N_Variables
str,str,str,str,u32
"""ESE""","""DE""","""04""","""Moneda y depós…",10
"""ESE""","""DE""","""04""","""Otros pasivos …",10
"""ESE""","""DE""","""04""","""Créditos y ant…",10
"""ESE""","""DE""","""04""","""Préstamos""",10
"""ESE""","""DE""","""04""","""Títulos de deu…",10
"""ESE""","""DE""","""04""","""Asignación de …",4
"""ESE""","""DE""","""04""","""Deuda externa …",1


Para ver todas las variables:


In [98]:
res_nivel_7(
    "ESE","DE",
    "Niv_Gr_3",
    "04",
    "Id","Niv_Descr_1","Niv_Descr_2","Niv_Descr_3").sort("Id")

<string>:29: FutureWarning:

Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.



Niv_Gr_1,Niv_Gr_2,Niv_Gr_3,Id,Niv_Descr_1,Niv_Descr_2,Niv_Descr_3,N_Variables
str,str,str,i64,str,str,str,u32
"""ESE""","""DE""","""04""",7392,"""Gobierno gener…","""A corto plazo …","""Asignación de …",1
"""ESE""","""DE""","""04""",7393,"""Gobierno gener…","""A corto plazo …","""Moneda y depós…",1
"""ESE""","""DE""","""04""",7394,"""Gobierno gener…","""A corto plazo …","""Títulos de deu…",1
"""ESE""","""DE""","""04""",7395,"""Gobierno gener…","""A corto plazo …","""Préstamos""",1
"""ESE""","""DE""","""04""",7396,"""Gobierno gener…","""A corto plazo …","""Créditos y ant…",1
…,…,…,…,…,…,…,…
"""ESE""","""DE""","""04""",7442,"""Sociedades no …","""A largo plazo …","""Títulos de deu…",1
"""ESE""","""DE""","""04""",7443,"""Sociedades no …","""A largo plazo …","""Préstamos""",1
"""ESE""","""DE""","""04""",7444,"""Sociedades no …","""A largo plazo …","""Créditos y ant…",1


**OJO: REVISAR LA CONSULTA A PARTIR DE 7392 HASTA 7446**

Resultados de una sola variable:


In [99]:
Id = 7447 # 7392 - 7446
df = get_df(Id)
# get_plot(Id)
df

<string>:29: FutureWarning:

Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.



<string>:29: FutureWarning:

Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.



Fecha,Consumo Intermedio-Actividades Inmobiliarias y Empresariales-Economía Total
date,f64
2000-01-01,1151.179
2001-01-01,1308.8859
2002-01-01,1405.3519
2003-01-01,1700.5176
2004-01-01,2002.3782
…,…
2015-01-01,5587.1
2016-01-01,6028.2
2017-01-01,6647.1


#### CEB = Comercio Exterior de Bienes


In [100]:
res_nivel_3("ESE","CEB","Niv_Gr_3")

4 subgrupos
339 variables


<string>:29: FutureWarning:

Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.



Niv_Gr_1,Niv_Gr_2,Niv_Gr_3,N_Variables
str,str,str,u32
"""ESE""","""CEB""","""04""",159
"""ESE""","""CEB""","""03""",93
"""ESE""","""CEB""","""01""",66
"""ESE""","""CEB""","""02""",21


##### 01 - Productos

* Banano
* Café
* Aceite de Palma
* Camarones
* Melones y Sandí­as
* Legumbres y Hortalizas
* Puros o Cigarros
* Textiles
* Jabones
* Plásticos y su Manufacturas
* Papel y Cartón


In [101]:
res_nivel_4(
    "ESE","CEB",
    "Niv_Gr_3",
    "01",
    "Niv_Descr_5")

<string>:29: FutureWarning:

Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.



Niv_Gr_1,Niv_Gr_2,Niv_Gr_3,Niv_Descr_5,N_Variables
str,str,str,str,u32
"""ESE""","""CEB""","""01""","""Banano""",6
"""ESE""","""CEB""","""01""","""Textiles""",6
"""ESE""","""CEB""","""01""","""Legumbres y Ho…",6
"""ESE""","""CEB""","""01""","""Papel y Cartón…",6
"""ESE""","""CEB""","""01""","""Café""",6
…,…,…,…,…
"""ESE""","""CEB""","""01""","""Plásticos y su…",6
"""ESE""","""CEB""","""01""","""Camarones""",6
"""ESE""","""CEB""","""01""","""Puros o Cigarr…",6


* Volumen
* Valor
* Precio


In [102]:
res_nivel_4(
    "ESE","CEB",
    "Niv_Gr_3",
    "01",
    "Niv_Descr_6")

<string>:29: FutureWarning:

Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.



Niv_Gr_1,Niv_Gr_2,Niv_Gr_3,Niv_Descr_6,N_Variables
str,str,str,str,u32
"""ESE""","""CEB""","""01""","""Precio""",22
"""ESE""","""CEB""","""01""","""Volumen""",22
"""ESE""","""CEB""","""01""","""Valor""",22


* Trimestral
* Anual


In [103]:
res_nivel_4(
    "ESE","CEB",
    "Niv_Gr_3",
    "01",
    "Niv_Descr_7")

<string>:29: FutureWarning:

Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.



Niv_Gr_1,Niv_Gr_2,Niv_Gr_3,Niv_Descr_7,N_Variables
str,str,str,str,u32
"""ESE""","""CEB""","""01""","""Anual""",33
"""ESE""","""CEB""","""01""","""Trimestral""",33


**OJO: "Niv_Descr_7" y "Niv_Descr_8" son iguales**

Para ver todas las variables:


In [104]:
res_nivel_7(
    "ESE","CEB",
    "Niv_Gr_3",
    "01","Id","Niv_Descr_5","Niv_Descr_6","Niv_Descr_7")

<string>:29: FutureWarning:

Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.



Niv_Gr_1,Niv_Gr_2,Niv_Gr_3,Id,Niv_Descr_5,Niv_Descr_6,Niv_Descr_7,N_Variables
str,str,str,i64,str,str,str,u32
"""ESE""","""CEB""","""01""",433,"""Camarones""","""Valor""","""Trimestral""",1
"""ESE""","""CEB""","""01""",449,"""Legumbres y Ho…","""Precio""","""Trimestral""",1
"""ESE""","""CEB""","""01""",456,"""Textiles""","""Valor""","""Anual""",1
"""ESE""","""CEB""","""01""",467,"""Jabones""","""Precio""","""Anual""",1
"""ESE""","""CEB""","""01""",427,"""Aceite de Palm…","""Valor""","""Trimestral""",1
…,…,…,…,…,…,…,…
"""ESE""","""CEB""","""01""",450,"""Puros o Cigarr…","""Valor""","""Trimestral""",1
"""ESE""","""CEB""","""01""",461,"""Textiles""","""Precio""","""Trimestral""",1
"""ESE""","""CEB""","""01""",470,"""Plásticos y su…","""Volumen""","""Trimestral""",1


Resultados de una sola variable:


In [105]:
Id = 441
df = get_df(Id)
# get_plot(Id)
df

<string>:29: FutureWarning:

Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.



<string>:29: FutureWarning:

Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.



Fecha,ESE-CEB-01-28-Melones y Sandías-Volumen-Trimestral-Trimestral
date,f64
2004-03-01,102961.3
2004-06-01,77639.6
2004-09-01,740.8
2004-12-01,15063.9
2005-03-01,108685.4
…,…
2022-12-01,3618.1
2023-03-01,123855.4
2023-06-01,71660.4


##### 02 - Combustibles

* Gasolina regular
* Bunker
* Diesel
* Total combustibles
* Gasolina superior
* Kerosene
* LPG


In [106]:
res_nivel_4(
    "ESE","CEB",
    "Niv_Gr_3",
    "02",
    "Niv_Descr_5")

<string>:29: FutureWarning:

Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.



Niv_Gr_1,Niv_Gr_2,Niv_Gr_3,Niv_Descr_5,N_Variables
str,str,str,str,u32
"""ESE""","""CEB""","""02""","""Bunker""",3
"""ESE""","""CEB""","""02""","""Diésel""",3
"""ESE""","""CEB""","""02""","""Kerosene""",3
"""ESE""","""CEB""","""02""","""LPG""",3
"""ESE""","""CEB""","""02""","""Gasolina regul…",3
"""ESE""","""CEB""","""02""","""Gasolina super…",3
"""ESE""","""CEB""","""02""","""Total combusti…",3


* Barriles
* Valor
* Precio


In [107]:
res_nivel_4(
    "ESE","CEB",
    "Niv_Gr_3",
    "02",
    "Niv_Descr_6")

<string>:29: FutureWarning:

Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.



Niv_Gr_1,Niv_Gr_2,Niv_Gr_3,Niv_Descr_6,N_Variables
str,str,str,str,u32
"""ESE""","""CEB""","""02""","""Valor""",7
"""ESE""","""CEB""","""02""","""Precio""",7
"""ESE""","""CEB""","""02""","""Barriles""",7


Para ver todas las variables:


In [108]:
res_nivel_7(
    "ESE","CEB",
    "Niv_Gr_3",
    "02","Id","Nombre","Niv_Descr_5","Niv_Descr_6")

<string>:29: FutureWarning:

Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.



Niv_Gr_1,Niv_Gr_2,Niv_Gr_3,Id,Nombre,Niv_Descr_5,Niv_Descr_6,N_Variables
str,str,str,i64,str,str,str,u32
"""ESE""","""CEB""","""02""",504,"""ESE-CEB-02-16""","""Total combusti…","""Valor""",1
"""ESE""","""CEB""","""02""",501,"""ESE-CEB-02-13""","""Total combusti…","""Precio""",1
"""ESE""","""CEB""","""02""",513,"""ESE-CEB-02-5""","""Kerosene""","""Barriles""",1
"""ESE""","""CEB""","""02""",516,"""ESE-CEB-02-8""","""LPG""","""Precio""",1
"""ESE""","""CEB""","""02""",498,"""ESE-CEB-02-10""","""Bunker""","""Precio""",1
…,…,…,…,…,…,…,…
"""ESE""","""CEB""","""02""",514,"""ESE-CEB-02-6""","""Bunker""","""Barriles""",1
"""ESE""","""CEB""","""02""",499,"""ESE-CEB-02-11""","""Gasolina regul…","""Precio""",1
"""ESE""","""CEB""","""02""",506,"""ESE-CEB-02-18""","""Bunker""","""Valor""",1


Resultados de una sola variable:


In [109]:
Id = 503
df = get_df(Id)
# get_plot(Id)
df

<string>:29: FutureWarning:

Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.



<string>:29: FutureWarning:

Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.



Fecha,ESE-CEB-02-15-Kerosene-Valor
date,f64
2016-01-01,1921.3
2016-02-01,1461.25
2016-03-01,3450.34
2016-04-01,1781.63
2016-05-01,3786.19
…,…
2023-08-01,9865.51
2023-09-01,6111.43
2023-10-01,4686.2


##### 03

* Importaciones AE 
* Importaciones BT Sección 
* Importaciones Secciones 


In [110]:
res_nivel_4(
    "ESE","CEB",
    "Niv_Gr_3",
    "03",
    "Niv_Descr_1")

<string>:29: FutureWarning:

Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.



Niv_Gr_1,Niv_Gr_2,Niv_Gr_3,Niv_Descr_1,N_Variables
str,str,str,str,u32
"""ESE""","""CEB""","""03""","""Importaciones …",50
"""ESE""","""CEB""","""03""","""Importaciones …",30
"""ESE""","""CEB""","""03""","""Importaciones …",13


Los dos primeros grupos solamente tienen un nivel de descripción:

###### Importaciones AE


In [111]:
res_nivel_7(
    "ESE","CEB",
    "Niv_Gr_3",
    "03","Id","Nombre","Niv_Descr_1","Niv_Descr_2"
    ).filter(
    pl.col("Niv_Descr_1") == "Importaciones AE ")

<string>:29: FutureWarning:

Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.



Niv_Gr_1,Niv_Gr_2,Niv_Gr_3,Id,Nombre,Niv_Descr_1,Niv_Descr_2,N_Variables
str,str,str,i64,str,str,str,u32
"""ESE""","""CEB""","""03""",6468,"""ESE-CEB-03-11""","""Importaciones …",""" 3. Combustibl…",1
"""ESE""","""CEB""","""03""",6476,"""ESE-CEB-03-19""","""Importaciones …",""" 5. Equipo de …",1
"""ESE""","""CEB""","""03""",6484,"""ESE-CEB-03-27""","""Importaciones …",""" 6.2 Semidurad…",1
"""ESE""","""CEB""","""03""",6485,"""ESE-CEB-03-28""","""Importaciones …",""" 6.3 No durade…",1
"""ESE""","""CEB""","""03""",6487,"""ESE-CEB-03-30""","""Importaciones …",""" 8. Todas las …",1
…,…,…,…,…,…,…,…
"""ESE""","""CEB""","""03""",6469,"""ESE-CEB-03-12""","""Importaciones …",""" 3.1 Sector Pr…",1
"""ESE""","""CEB""","""03""",6470,"""ESE-CEB-03-13""","""Importaciones …",""" 3.2 Procesado…",1
"""ESE""","""CEB""","""03""",6474,"""ESE-CEB-03-17""","""Importaciones …",""" 4.1 Bienes de…",1


###### Importaciones BT Sección 


In [112]:
res_nivel_7(
    "ESE","CEB",
    "Niv_Gr_3",
    "03","Id","Nombre","Niv_Descr_1","Niv_Descr_2"
    ).filter(
    pl.col("Niv_Descr_1") == "Importaciones BT Sección ")

<string>:29: FutureWarning:

Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.



Niv_Gr_1,Niv_Gr_2,Niv_Gr_3,Id,Nombre,Niv_Descr_1,Niv_Descr_2,N_Variables
str,str,str,i64,str,str,str,u32
"""ESE""","""CEB""","""03""",6496,"""ESE-CEB-03-39""","""Importaciones …",""" 16 Máquinas y…",1
"""ESE""","""CEB""","""03""",6490,"""ESE-CEB-03-33""","""Importaciones …",""" 7 Plástico y …",1
"""ESE""","""CEB""","""03""",6498,"""ESE-CEB-03-41""","""Importaciones …",""" 18 Instrumene…",1
"""ESE""","""CEB""","""03""",6494,"""ESE-CEB-03-37""","""Importaciones …",""" 13 Manufactur…",1
"""ESE""","""CEB""","""03""",6499,"""ESE-CEB-03-42""","""Importaciones …",""" 20 Mercancías…",1
…,…,…,…,…,…,…,…
"""ESE""","""CEB""","""03""",6488,"""ESE-CEB-03-31""","""Importaciones …",""" 5 Productos m…",1
"""ESE""","""CEB""","""03""",6492,"""ESE-CEB-03-35""","""Importaciones …",""" 10 Pasta de M…",1
"""ESE""","""CEB""","""03""",6495,"""ESE-CEB-03-38""","""Importaciones …",""" 15 Metales co…",1


El tercer grupo tiene dos niveles de descripción

###### Importaciones Secciones 

* Valores en Millones de USD
* Peso en Kilos


In [113]:
res_nivel_7(
    "ESE","CEB",
    "Niv_Gr_3",
    "03","Id","Niv_Descr_1","Niv_Descr_2","Niv_Descr_3"
    ).filter(
    pl.col("Niv_Descr_1") == "Importaciones Secciones ")

<string>:29: FutureWarning:

Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.



Niv_Gr_1,Niv_Gr_2,Niv_Gr_3,Id,Niv_Descr_1,Niv_Descr_2,Niv_Descr_3,N_Variables
str,str,str,i64,str,str,str,u32
"""ESE""","""CEB""","""03""",6534,"""Importaciones …",""" Peso en Kilos…",""" 7 Plástico y …",1
"""ESE""","""CEB""","""03""",6547,"""Importaciones …",""" Peso en Kilos…",""" 20 Mercancía …",1
"""ESE""","""CEB""","""03""",6506,"""Importaciones …",""" Valores en Mi…",""" 5.1 Combustib…",1
"""ESE""","""CEB""","""03""",6531,"""Importaciones …",""" Peso en Kilos…",""" 5.1 Combustib…",1
"""ESE""","""CEB""","""03""",6539,"""Importaciones …",""" Peso en Kilos…",""" 12 Calzado, s…",1
…,…,…,…,…,…,…,…
"""ESE""","""CEB""","""03""",6522,"""Importaciones …",""" Valores en Mi…",""" 20 Mercancía …",1
"""ESE""","""CEB""","""03""",6501,"""Importaciones …",""" Valores en Mi…",""" 1 Animales vi…",1
"""ESE""","""CEB""","""03""",6507,"""Importaciones …",""" Valores en Mi…",""" 5.2 Otros pro…",1


Para ver todas las variables:


In [114]:
res_nivel_7(
    "ESE","CEB",
    "Niv_Gr_3",
    "03","Id","Niv_Descr_1","Niv_Descr_2","Niv_Descr_3"
    )

<string>:29: FutureWarning:

Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.



Niv_Gr_1,Niv_Gr_2,Niv_Gr_3,Id,Niv_Descr_1,Niv_Descr_2,Niv_Descr_3,N_Variables
str,str,str,i64,str,str,str,u32
"""ESE""","""CEB""","""03""",6483,"""Importaciones …",""" 6.1 Duraderos…",null,1
"""ESE""","""CEB""","""03""",6491,"""Importaciones …",""" 9 Madera, car…",null,1
"""ESE""","""CEB""","""03""",6496,"""Importaciones …",""" 16 Máquinas y…",null,1
"""ESE""","""CEB""","""03""",6518,"""Importaciones …",""" Valores en Mi…",""" 16 Máquinas y…",1
"""ESE""","""CEB""","""03""",6463,"""Importaciones …",""" 1.2.1 Princip…",null,1
…,…,…,…,…,…,…,…
"""ESE""","""CEB""","""03""",6532,"""Importaciones …",""" Peso en Kilos…",""" 5.2 Otros pro…",1
"""ESE""","""CEB""","""03""",6538,"""Importaciones …",""" Peso en Kilos…",""" 11 Materias t…",1
"""ESE""","""CEB""","""03""",6542,"""Importaciones …",""" Peso en Kilos…",""" 15 Metales co…",1


Resultados de una sola variable:


In [115]:
Id = 6517
df = get_df(Id)
# get_plot(Id)
df

<string>:29: FutureWarning:

Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.



<string>:29: FutureWarning:

Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.



Fecha,Importaciones Secciones - Valores en Millones de USD - 15 Metales comunes y sus manufacturas
date,f64
2021-01-01,75.2166
2021-02-01,72.4471
2021-03-01,93.2076
2021-04-01,72.5089
2021-05-01,120.7819
…,…
2023-08-01,95.8238
2023-09-01,114.9642
2023-10-01,86.7132


##### 04 AE y BT

* Exportaciones AE
* Exportaciones BT


In [116]:
res_nivel_4(
    "ESE","CEB",
    "Niv_Gr_3",
    "04",
    "Niv_Descr_1")

<string>:29: FutureWarning:

Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.



Niv_Gr_1,Niv_Gr_2,Niv_Gr_3,Niv_Descr_1,N_Variables
str,str,str,str,u32
"""ESE""","""CEB""","""04""","""Exportaciones …",137
"""ESE""","""CEB""","""04""","""Exportaciones …",22


* Producto
* Grupo Producto


In [117]:
res_nivel_4(
    "ESE","CEB",
    "Niv_Gr_3",
    "04",
    "Niv_Descr_2")

<string>:29: FutureWarning:

Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.



Niv_Gr_1,Niv_Gr_2,Niv_Gr_3,Niv_Descr_2,N_Variables
str,str,str,str,u32
"""ESE""","""CEB""","""04""",""" Producto """,137
"""ESE""","""CEB""","""04""","""Grupo Producto…",22


Para ver todas las variables:


In [118]:
res_nivel_7(
    "ESE","CEB",
    "Niv_Gr_3",
    "04","Id","Niv_Descr_1","Niv_Descr_2","Niv_Descr_3")

<string>:29: FutureWarning:

Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.



Niv_Gr_1,Niv_Gr_2,Niv_Gr_3,Id,Niv_Descr_1,Niv_Descr_2,Niv_Descr_3,N_Variables
str,str,str,i64,str,str,str,u32
"""ESE""","""CEB""","""04""",6563,"""Exportaciones …",""" Producto """,""" 1.12 Otras fr…",1
"""ESE""","""CEB""","""04""",6571,"""Exportaciones …",""" Producto """,""" 1.20 Plátano""",1
"""ESE""","""CEB""","""04""",6576,"""Exportaciones …",""" Producto """,""" 1.25 Resinas …",1
"""ESE""","""CEB""","""04""",6580,"""Exportaciones …",""" Producto """,""" 1.29 Sorgo""",1
"""ESE""","""CEB""","""04""",6615,"""Exportaciones …",""" Producto """,""" 3.28 Cemento,…",1
…,…,…,…,…,…,…,…
"""ESE""","""CEB""","""04""",6633,"""Exportaciones …",""" Producto """,""" 3.46 Harina d…",1
"""ESE""","""CEB""","""04""",6660,"""Exportaciones …",""" Producto """,""" 3.73 Maquinar…",1
"""ESE""","""CEB""","""04""",6675,"""Exportaciones …",""" Producto """,""" 3.89 Bebidas …",1


Resultados de una sola variable:


In [119]:
Id = 6554
df = get_df(Id)
# get_plot(Id)
df

<string>:29: FutureWarning:

Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.



<string>:29: FutureWarning:

Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.



Fecha,"Exportaciones AE- Producto - 1.3 Otras hortalizas, legumbres y demás vegetales y semillas de hortalizas"
date,f64
2021-01-01,6049.6537
2021-02-01,6190.9441
2021-03-01,7928.1926
2021-04-01,5373.3219
2021-05-01,6966.3106
…,…
2023-08-01,2825.6754
2023-09-01,1473.4386
2023-10-01,2589.7748


#### BP = Balanza de Pagos


In [120]:
res_nivel_3("ESE","BP","Niv_Gr_3")

1 subgrupos
182 variables


<string>:29: FutureWarning:

Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.



Niv_Gr_1,Niv_Gr_2,Niv_Gr_3,N_Variables
str,str,str,u32
"""ESE""","""BP""","""01""",182


* Anual
* Mensual


In [121]:
res_nivel_4(
    "ESE","BP",
    "Niv_Gr_3",
    "01",
    "Niv_Descr_2")

<string>:29: FutureWarning:

Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.



Niv_Gr_1,Niv_Gr_2,Niv_Gr_3,Niv_Descr_2,N_Variables
str,str,str,str,u32
"""ESE""","""BP""","""01""",""" Anual""",91
"""ESE""","""BP""","""01""",""" Mensual""",91


Por Componentes:


In [122]:
res_nivel_4(
    "ESE","BP",
    "Niv_Gr_3",
    "01",
    "Niv_Descr_1")

<string>:29: FutureWarning:

Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.



Niv_Gr_1,Niv_Gr_2,Niv_Gr_3,Niv_Descr_1,N_Variables
str,str,str,str,u32
"""ESE""","""BP""","""01""","""1. Ingresos """,4
"""ESE""","""BP""","""01""","""2.2 Bienes par…",2
"""ESE""","""BP""","""01""","""1.2 Otra Renta…",2
"""ESE""","""BP""","""01""","""3.1.4.1 Banco …",2
"""ESE""","""BP""","""01""","""3.2 Pasivos """,2
…,…,…,…,…
"""ESE""","""BP""","""01""","""1.1 Inversión …",2
"""ESE""","""BP""","""01""","""3.1.2.2 Gobier…",2
"""ESE""","""BP""","""01""","""1.1 Transporte…",2


Para ver todas las variables:


In [123]:
res_nivel_7(
    "ESE","BP",
    "Niv_Gr_3",
    "01","Id","Nombre","Niv_Descr_2","Niv_Descr_1").sort("Niv_Descr_2")

<string>:29: FutureWarning:

Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.



Niv_Gr_1,Niv_Gr_2,Niv_Gr_3,Id,Nombre,Niv_Descr_2,Niv_Descr_1,N_Variables
str,str,str,i64,str,str,str,u32
"""ESE""","""BP""","""01""",6384,"""ESE-BP-01-109""",""" Anual""","""2.2 Viajes """,1
"""ESE""","""BP""","""01""",6435,"""ESE-BP-01-160""",""" Anual""","""3.2.1.1 Gobier…",1
"""ESE""","""BP""","""01""",6436,"""ESE-BP-01-161""",""" Anual""","""3.2.1.2 Otros …",1
"""ESE""","""BP""","""01""",6447,"""ESE-BP-01-172""",""" Anual""","""3.2.4 Otros Pa…",1
"""ESE""","""BP""","""01""",6406,"""ESE-BP-01-131""",""" Anual""","""1.2 Inversión …",1
…,…,…,…,…,…,…,…
"""ESE""","""BP""","""01""",6337,"""ESE-BP-01-62""",""" Mensual""","""3.1.4 Otros A…",1
"""ESE""","""BP""","""01""",6344,"""ESE-BP-01-69""",""" Mensual""","""3.2.1.1 Gobier…",1
"""ESE""","""BP""","""01""",6355,"""ESE-BP-01-80""",""" Mensual""","""3.2.3.4 Otros …",1


Resultados de una sola variable:


In [124]:
Id = 6292
df = get_df(Id)
# get_plot(Id)
df

<string>:29: FutureWarning:

Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.



<string>:29: FutureWarning:

Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.



Fecha,2.1 Transporte - Mensual
date,f64
2004-03-01,98.1
2004-06-01,104.5
2004-09-01,113.6
2004-12-01,123.2
2005-03-01,116.1
…,…
2022-12-01,581.3
2023-03-01,461.7
2023-06-01,477.0


#### IES - Ingresos y Egresos de Servicios


In [125]:
res_nivel_3("ESE","IES","Niv_Gr_3")

1 subgrupos
28 variables


<string>:29: FutureWarning:

Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.



Niv_Gr_1,Niv_Gr_2,Niv_Gr_3,N_Variables
str,str,str,u32
"""ESE""","""IES""","""01""",28


Periodicidad:

* Anual
* Trimestral


In [126]:
res_nivel_4(
    "ESE","IES",
    "Niv_Descr_3",
    "01","Niv_Descr_5")

<string>:29: FutureWarning:

Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.



Niv_Gr_1,Niv_Gr_2,Niv_Descr_3,Niv_Descr_5,N_Variables
str,str,str,str,u32
"""ESE""","""IES""","""01""","""Trimestral""",14
"""ESE""","""IES""","""01""","""Anual""",14


Ingresos y Egresos:

* Exportaciones (Ingresos)
* Importaciones (Egresos)


In [127]:
res_nivel_4(
    "ESE","IES",
    "Niv_Descr_3",
    "01","Niv_Descr_6")

<string>:29: FutureWarning:

Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.



Niv_Gr_1,Niv_Gr_2,Niv_Descr_3,Niv_Descr_6,N_Variables
str,str,str,str,u32
"""ESE""","""IES""","""01""","""Importaciones …",16
"""ESE""","""IES""","""01""","""Exportaciones …",12


Variables:

* Transporte
* Viajes
* Servicios de Comunicaciones
* Otros Servicios Empresariales
* Servicios de Gobierno
* Otros Servicios
* Servicios de Seguro
* Servicios de InformÃ¡tica e Información
* Servicios de RegalÃ­as y Derechos de Licencia


In [128]:
res_nivel_4(
    "ESE","IES",
    "Niv_Descr_3",
    "01","Niv_Descr_7")

<string>:29: FutureWarning:

Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.



Niv_Gr_1,Niv_Gr_2,Niv_Descr_3,Niv_Descr_7,N_Variables
str,str,str,str,u32
"""ESE""","""IES""","""01""","""Otros Servicio…",4
"""ESE""","""IES""","""01""","""Servicios de C…",4
"""ESE""","""IES""","""01""","""Transporte""",4
"""ESE""","""IES""","""01""","""Otros Servicio…",4
"""ESE""","""IES""","""01""","""Viajes""",4
"""ESE""","""IES""","""01""","""Servicios de S…",2
"""ESE""","""IES""","""01""","""Servicios de G…",2
"""ESE""","""IES""","""01""","""Servicios de R…",2
"""ESE""","""IES""","""01""","""Servicios de I…",2


Para ver todas las variables:


In [129]:
res_nivel_7(
    "ESE","IES",
    "Niv_Gr_3",
    "01","Id","Niv_Descr_5","Niv_Descr_6","Niv_Descr_7")

<string>:29: FutureWarning:

Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.



Niv_Gr_1,Niv_Gr_2,Niv_Gr_3,Id,Niv_Descr_5,Niv_Descr_6,Niv_Descr_7,N_Variables
str,str,str,i64,str,str,str,u32
"""ESE""","""IES""","""01""",399,"""Trimestral""","""Importaciones …","""Servicios de S…",1
"""ESE""","""IES""","""01""",392,"""Trimestral""","""Exportaciones …","""Servicios de C…",1
"""ESE""","""IES""","""01""",385,"""Anual""","""Importaciones …","""Servicios de S…",1
"""ESE""","""IES""","""01""",380,"""Anual""","""Exportaciones …","""Servicios de G…",1
"""ESE""","""IES""","""01""",386,"""Anual""","""Importaciones …","""Servicios de I…",1
…,…,…,…,…,…,…,…
"""ESE""","""IES""","""01""",379,"""Anual""","""Exportaciones …","""Otros Servicio…",1
"""ESE""","""IES""","""01""",401,"""Trimestral""","""Importaciones …","""Servicios de R…",1
"""ESE""","""IES""","""01""",395,"""Trimestral""","""Exportaciones …","""Otros Servicio…",1


Resultados de una sola variable:


In [130]:
Id = 388
df = get_df(Id)
# get_plot(Id)
df

<string>:29: FutureWarning:

Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.



<string>:29: FutureWarning:

Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.



Fecha,ESE-IES-01-13-Anual-Importaciones (Egresos)-Otros Servicios Empresariales
date,f64
2000-01-01,46.2
2001-01-01,57.0
2002-01-01,48.1
2003-01-01,48.3
2004-01-01,50.7
…,…
2019-01-01,236.8
2020-01-01,265.5
2021-01-01,227.2


#### PII = Posición de Inversión Inernacional


In [131]:
res_nivel_3("ESE","PII","Niv_Gr_3")

1 subgrupos
17 variables


<string>:29: FutureWarning:

Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.



Niv_Gr_1,Niv_Gr_2,Niv_Gr_3,N_Variables
str,str,str,u32
"""ESE""","""PII""","""01""",17


Agrupación:

* Posición de Inversión Internacional por Categorí­a  
* Posición de los Activos Financieros con no Residentes por Componente 
* Posición de los Pasivos Financieros con no Residentes por Componente 
* Posición de Inversión Internacional Neta por Componente 


In [132]:
res_nivel_4(
    "ESE","PII",
    "Niv_Gr_3",
    "01","Niv_Descr_5")

<string>:29: FutureWarning:

Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.



Niv_Gr_1,Niv_Gr_2,Niv_Gr_3,Niv_Descr_5,N_Variables
str,str,str,str,u32
"""ESE""","""PII""","""01""",""" Posición de I…",5
"""ESE""","""PII""","""01""",""" Posición de l…",5
"""ESE""","""PII""","""01""",""" Posición de l…",4
"""ESE""","""PII""","""01""",""" Posición de I…",3


Cuentas:

* Activos 
* Pasivos
* PII Neta
* Inversió³n extranjera directa
* Inversión de cartera
* Derivados financieros
* Otra inversión
* Activos de reservas lí­quidos oficiales


In [133]:
res_nivel_4(
    "ESE","PII",
    "Niv_Gr_3",
    "01","Niv_Descr_6")

<string>:29: FutureWarning:

Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.



Niv_Gr_1,Niv_Gr_2,Niv_Gr_3,Niv_Descr_6,N_Variables
str,str,str,str,u32
"""ESE""","""PII""","""01""",""" Inversión ext…",3
"""ESE""","""PII""","""01""",""" Otra inversió…",3
"""ESE""","""PII""","""01""",""" Derivados fin…",3
"""ESE""","""PII""","""01""",""" Inversión de …",3
"""ESE""","""PII""","""01""",""" Activos de re…",2
"""ESE""","""PII""","""01""",""" PII Neta""",1
"""ESE""","""PII""","""01""",""" Pasivos""",1
"""ESE""","""PII""","""01""",""" Activos """,1


Para ver todas las variables:


In [134]:
res_nivel_7(
    "ESE","PII",
    "Niv_Gr_3",
    "01","Id","Nombre","Niv_Descr_5","Niv_Descr_6")

<string>:29: FutureWarning:

Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.



Niv_Gr_1,Niv_Gr_2,Niv_Gr_3,Id,Nombre,Niv_Descr_5,Niv_Descr_6,N_Variables
str,str,str,i64,str,str,str,u32
"""ESE""","""PII""","""01""",493,"""ESE-PII-01-A14…",""" Posición de I…",""" Inversión de …",1
"""ESE""","""PII""","""01""",496,"""ESE-PII-01-A17…",""" Posición de I…",""" Activos de re…",1
"""ESE""","""PII""","""01""",484,"""ESE-PII-01-A5""",""" Posición de l…",""" Inversión de …",1
"""ESE""","""PII""","""01""",490,"""ESE-PII-01-A11…",""" Posición de l…",""" Derivados fin…",1
"""ESE""","""PII""","""01""",480,"""ESE-PII-01-A1""",""" Posición de I…",""" Activos """,1
…,…,…,…,…,…,…,…
"""ESE""","""PII""","""01""",489,"""ESE-PII-01-A10…",""" Posición de l…",""" Inversión de …",1
"""ESE""","""PII""","""01""",483,"""ESE-PII-01-A4""",""" Posición de l…",""" Inversión ext…",1
"""ESE""","""PII""","""01""",488,"""ESE-PII-01-A9""",""" Posición de l…",""" Inversión ext…",1


Resultados de una sola variable:


In [135]:
Id = 496
df = get_df(Id)
# get_plot(Id)
df

<string>:29: FutureWarning:

Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.



<string>:29: FutureWarning:

Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.



Fecha,ESE-PII-01 - D - Posición de Inversión Internacional Neta por Componente - Activos de reservas líquidos oficiales
date,f64
2008-03-01,2512.7
2008-06-01,2541.4
2008-09-01,2350.1
2008-12-01,2449.7
2009-03-01,2542.6
…,…
2022-12-01,8410.5
2023-03-01,8122.5
2023-06-01,7902.2


#### IED = Inversión Extranjera Directa


In [136]:
res_nivel_3("ESE","IED","Niv_Gr_3")

1 subgrupos
10 variables


<string>:29: FutureWarning:

Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.



Niv_Gr_1,Niv_Gr_2,Niv_Gr_3,N_Variables
str,str,str,u32
"""ESE""","""IED""","""01""",10


Para ver todas las variables:


In [137]:
res_nivel_6(
    "ESE","IED",
    "Niv_Gr_3",
    "01","Id","Nombre","Niv_Descr_4")

<string>:29: FutureWarning:

Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.



Niv_Gr_1,Niv_Gr_2,Niv_Gr_3,Id,Nombre,Niv_Descr_4,N_Variables
str,str,str,i64,str,str,u32
"""ESE""","""IED""","""01""",406,"""ESE-IED-01-2""",""" Comercio, Res…",1
"""ESE""","""IED""","""01""",405,"""ESE-IED-01-10""",""" Transporte, A…",1
"""ESE""","""IED""","""01""",410,"""ESE-IED-01-6""",""" Industria Man…",1
"""ESE""","""IED""","""01""",413,"""ESE-IED-01-9""",""" IED Total""",1
"""ESE""","""IED""","""01""",407,"""ESE-IED-01-3""",""" Construcción""",1
"""ESE""","""IED""","""01""",411,"""ESE-IED-01-7""",""" Minas y Cante…",1
"""ESE""","""IED""","""01""",409,"""ESE-IED-01-5""",""" Industria de …",1
"""ESE""","""IED""","""01""",408,"""ESE-IED-01-4""",""" Electricidad,…",1
"""ESE""","""IED""","""01""",412,"""ESE-IED-01-8""",""" Servicios""",1


Resultados de una sola variable:


In [138]:
Id = 409
df = get_df(Id)
# get_plot(Id)
df

<string>:29: FutureWarning:

Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.



<string>:29: FutureWarning:

Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.



Fecha,ESE-IED-01 - Industria de Bienes para Transformación (Maquila)
date,f64
2000-01-01,50.7
2001-01-01,73.9
2002-01-01,73.3
2003-01-01,108.8
2004-01-01,174.9
…,…
2019-01-01,-34.6
2020-01-01,235.0
2021-01-01,77.4


### EMF = Estadísiticas Monetarias y Financieras


In [139]:
res_nivel_2("EMF")

11519 variables
54 grupos
12 variables
4 grupos
12 subgrupos


<string>:29: FutureWarning:

Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.



Niv_Gr_1,Niv_Gr_2,N_Variables,N_Niv_Gr_2
str,str,u32,u32
"""EMF""","""AMCC""",6,6
"""EMF""","""TI""",3,3
"""EMF""","""PFS""",2,2
"""EMF""","""EBM""",1,1


#### AMCC = Agregados Monetarios, Crédito y Captación


In [140]:
res_nivel_3("EMF","AMCC","Niv_Gr_3").sort("Niv_Gr_3")

6 subgrupos
102 variables


<string>:29: FutureWarning:

Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.



Niv_Gr_1,Niv_Gr_2,Niv_Gr_3,N_Variables
str,str,str,u32
"""EMF""","""AMCC""","""01""",46
"""EMF""","""AMCC""","""02""",16
"""EMF""","""AMCC""","""03""",6
"""EMF""","""AMCC""","""04""",6
"""EMF""","""AMCC""","""05""",14
"""EMF""","""AMCC""","""06""",14


##### 01 Agregados Monetarios, Crédito y Captación

Grupos:


In [141]:
res_nivel_4(
    "EMF","AMCC",
    "Niv_Gr_3","01",
    "Niv_Descr_5")

<string>:29: FutureWarning:

Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.



Niv_Gr_1,Niv_Gr_2,Niv_Gr_3,Niv_Descr_5,N_Variables
str,str,str,str,u32
"""EMF""","""AMCC""","""01""",""" Captación Ban…",8
"""EMF""","""AMCC""","""01""",""" Captación Otr…",8
"""EMF""","""AMCC""","""01""",""" Crédito Otras…",6
"""EMF""","""AMCC""","""01""",""" Crédito de lo…",6
"""EMF""","""AMCC""","""01""",""" Agregados Mon…",6
"""EMF""","""AMCC""","""01""",""" Crédito Secto…",6
"""EMF""","""AMCC""","""01""",""" Crédito Otras…",6


Variables:


In [142]:
res_nivel_4(
    "EMF","AMCC",
    "Niv_Gr_3","01",
    "Niv_Descr_6")

<string>:29: FutureWarning:

Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.



Niv_Gr_1,Niv_Gr_2,Niv_Gr_3,Niv_Descr_6,N_Variables
str,str,str,str,u32
"""EMF""","""AMCC""","""01""",""" Hogares""",12
"""EMF""","""AMCC""","""01""",""" Total""",12
"""EMF""","""AMCC""","""01""",""" Empresas Priv…",12
"""EMF""","""AMCC""","""01""",""" Otros Sectore…",4
"""EMF""","""AMCC""","""01""",""" M3""",2
"""EMF""","""AMCC""","""01""",""" M2""",2
"""EMF""","""AMCC""","""01""",""" M1""",2


Para ver todas las variables:


In [143]:
res_nivel_7(
    "EMF","AMCC",
    "Niv_Gr_3","01",
    "Id","Niv_Descr_5","Niv_Descr_6","Niv_Descr_4")

<string>:29: FutureWarning:

Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.



Niv_Gr_1,Niv_Gr_2,Niv_Gr_3,Id,Niv_Descr_5,Niv_Descr_6,Niv_Descr_4,N_Variables
str,str,str,i64,str,str,str,u32
"""EMF""","""AMCC""","""01""",545,""" Captación Otr…",""" Total""",""" VariaciónCapt…",1
"""EMF""","""AMCC""","""01""",551,""" Captación Ban…",""" Total""",""" Captacióndelo…",1
"""EMF""","""AMCC""","""01""",564,""" Crédito de lo…",""" Empresas Priv…",""" VariaciónCréd…",1
"""EMF""","""AMCC""","""01""",579,""" Captación Ban…",""" Otros Sectore…",""" VariaciónCapt…",1
"""EMF""","""AMCC""","""01""",534,""" Agregados Mon…",""" M1""",""" VariaciónM1 """,1
…,…,…,…,…,…,…,…
"""EMF""","""AMCC""","""01""",542,""" Crédito de lo…",""" Total""",""" VariaciónCréd…",1
"""EMF""","""AMCC""","""01""",544,""" Crédito Otras…",""" Total""",""" CréditodelasO…",1
"""EMF""","""AMCC""","""01""",576,""" Captación Ban…",""" Otros Sectore…",""" CaptaciónBCOt…",1


Resultados de una sola variable:


In [144]:
Id = 548
df = get_df(Id)
# get_plot(Id)
df

<string>:29: FutureWarning:

Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.



<string>:29: FutureWarning:

Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.



Fecha,EMF-AMCC-01 - CaptacióndelasOSDTotal - Captación Otras Sociedades de Depósito - Total
date,f64
2009-01-01,121098.9568
2009-02-01,122857.7616
2009-03-01,123678.236
2009-04-01,124122.7985
2009-05-01,124697.6893
…,…
2023-11-01,545201.5847
2023-12-01,559471.0519
2024-01-01,562707.5954


##### 02 Captación: Bancos Comerciales y Otras Sociedades de Depósito (Variación y Saldo)


In [145]:
res_nivel_4(
    "EMF","AMCC",
    "Niv_Gr_3","02",
    "Niv_Descr_1")

<string>:29: FutureWarning:

Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.



Niv_Gr_1,Niv_Gr_2,Niv_Gr_3,Niv_Descr_1,N_Variables
str,str,str,str,u32
"""EMF""","""AMCC""","""02""","""Captación Otra…",8
"""EMF""","""AMCC""","""02""","""Captación Banc…",8


In [146]:
res_nivel_4(
    "EMF","AMCC",
    "Niv_Gr_3","02",
    "Niv_Descr_3")

<string>:29: FutureWarning:

Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.



Niv_Gr_1,Niv_Gr_2,Niv_Gr_3,Niv_Descr_3,N_Variables
str,str,str,str,u32
"""EMF""","""AMCC""","""02""","""Hogares""",4
"""EMF""","""AMCC""","""02""","""Total""",4
"""EMF""","""AMCC""","""02""","""Otros Sectores…",4
"""EMF""","""AMCC""","""02""","""Empresas Priva…",4


In [147]:
res_nivel_4(
    "EMF","AMCC",
    "Niv_Gr_3","02",
    "Niv_Descr_4")

<string>:29: FutureWarning:

Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.



Niv_Gr_1,Niv_Gr_2,Niv_Gr_3,Niv_Descr_4,N_Variables
str,str,str,str,u32
"""EMF""","""AMCC""","""02""","""Saldo""",8
"""EMF""","""AMCC""","""02""","""Variación""",8


Para ver todas las variables:


In [148]:
res_nivel_7(
    "EMF","AMCC",
    "Niv_Gr_3","02",
    "Id","Niv_Descr_1","Niv_Descr_3","Niv_Descr_4")

<string>:29: FutureWarning:

Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.



Niv_Gr_1,Niv_Gr_2,Niv_Gr_3,Id,Niv_Descr_1,Niv_Descr_3,Niv_Descr_4,N_Variables
str,str,str,i64,str,str,str,u32
"""EMF""","""AMCC""","""02""",626,"""Captación Banc…","""Empresas Priva…","""Variación""",1
"""EMF""","""AMCC""","""02""",622,"""Captación Banc…","""Empresas Priva…","""Saldo""",1
"""EMF""","""AMCC""","""02""",629,"""Captación Otra…","""Total""","""Saldo""",1
"""EMF""","""AMCC""","""02""",631,"""Captación Otra…","""Empresas Priva…","""Saldo""",1
"""EMF""","""AMCC""","""02""",633,"""Captación Otra…","""Total""","""Variación""",1
…,…,…,…,…,…,…,…
"""EMF""","""AMCC""","""02""",635,"""Captación Otra…","""Empresas Priva…","""Variación""",1
"""EMF""","""AMCC""","""02""",621,"""Captación Banc…","""Hogares""","""Saldo""",1
"""EMF""","""AMCC""","""02""",636,"""Captación Otra…","""Otros Sectores…","""Variación""",1


Resultados de una sola variable:


In [149]:
Id = 622
df = get_df(Id)
# get_plot(Id)
df

<string>:29: FutureWarning:

Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.



<string>:29: FutureWarning:

Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.



Fecha,Captación Bancos Comerciales-CaptaciónBCOtrasSociedadesnoFinancieras-Empresas Privadas-Saldo
date,f64
2009-01-01,34027.9427
2009-02-01,35448.6052
2009-03-01,35268.8324
2009-04-01,36897.5217
2009-05-01,36938.7292
…,…
2023-11-01,155805.7389
2023-12-01,159703.4382
2024-01-01,164128.4025


##### 03 Crédito de los Bancos Comerciales


In [150]:
res_nivel_4(
    "EMF","AMCC",
    "Niv_Gr_3","03",
    "Niv_Descr_1")

<string>:29: FutureWarning:

Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.



Niv_Gr_1,Niv_Gr_2,Niv_Gr_3,Niv_Descr_1,N_Variables
str,str,str,str,u32
"""EMF""","""AMCC""","""03""","""Crédito de los…",6


In [151]:
res_nivel_4(
    "EMF","AMCC",
    "Niv_Gr_3","03",
    "Niv_Descr_4")

<string>:29: FutureWarning:

Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.



Niv_Gr_1,Niv_Gr_2,Niv_Gr_3,Niv_Descr_4,N_Variables
str,str,str,str,u32
"""EMF""","""AMCC""","""03""","""Variación""",3
"""EMF""","""AMCC""","""03""","""Saldo""",3


In [152]:
res_nivel_4(
    "EMF","AMCC",
    "Niv_Gr_3","03",
    "Niv_Descr_3")

<string>:29: FutureWarning:

Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.



Niv_Gr_1,Niv_Gr_2,Niv_Gr_3,Niv_Descr_3,N_Variables
str,str,str,str,u32
"""EMF""","""AMCC""","""03""","""Empresas Priva…",2
"""EMF""","""AMCC""","""03""","""Total""",2
"""EMF""","""AMCC""","""03""","""Hogares""",2


Para ver todas las variables:


In [153]:
res_nivel_7(
    "EMF","AMCC",
    "Niv_Gr_3","03",
    "Id","Niv_Descr_1","Niv_Descr_3","Niv_Descr_4")

<string>:29: FutureWarning:

Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.



Niv_Gr_1,Niv_Gr_2,Niv_Gr_3,Id,Niv_Descr_1,Niv_Descr_3,Niv_Descr_4,N_Variables
str,str,str,i64,str,str,str,u32
"""EMF""","""AMCC""","""03""",638,"""Crédito de los…","""Hogares""","""Saldo""",1
"""EMF""","""AMCC""","""03""",640,"""Crédito de los…","""Empresas Priva…","""Variación""",1
"""EMF""","""AMCC""","""03""",642,"""Crédito de los…","""Total""","""Variación""",1
"""EMF""","""AMCC""","""03""",639,"""Crédito de los…","""Total""","""Saldo""",1
"""EMF""","""AMCC""","""03""",637,"""Crédito de los…","""Empresas Priva…","""Saldo""",1
"""EMF""","""AMCC""","""03""",641,"""Crédito de los…","""Hogares""","""Variación""",1


Resultados de una sola variable:


In [154]:
Id = 638
df = get_df(Id)
# get_plot(Id)
df

<string>:29: FutureWarning:

Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.



<string>:29: FutureWarning:

Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.



Fecha,Crédito de los Bancos Comerciales-CréditoBCHogares-Hogares-Saldo
date,f64
2009-01-01,55678.9931
2009-02-01,54470.0449
2009-03-01,55864.692
2009-04-01,56912.6855
2009-05-01,56863.5362
…,…
2023-11-01,245899.9323
2023-12-01,250488.4006
2024-01-01,253037.8958


##### 04 Crédito Otras Sociedades de Depósito


In [155]:
res_nivel_4(
    "EMF","AMCC",
    "Niv_Gr_3","04",
    "Niv_Descr_1")

<string>:29: FutureWarning:

Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.



Niv_Gr_1,Niv_Gr_2,Niv_Gr_3,Niv_Descr_1,N_Variables
str,str,str,str,u32
"""EMF""","""AMCC""","""04""","""Crédito Otras …",6


In [156]:
res_nivel_5(
    "EMF","AMCC",
    "Niv_Gr_3","04",
    "Niv_Descr_3","Niv_Descr_4")

<string>:29: FutureWarning:

Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.



Niv_Gr_1,Niv_Gr_2,Niv_Gr_3,Niv_Descr_3,Niv_Descr_4,N_Variables
str,str,str,str,str,u32
"""EMF""","""AMCC""","""04""","""Hogares""","""Saldo""",1
"""EMF""","""AMCC""","""04""","""Total""","""Saldo""",1
"""EMF""","""AMCC""","""04""","""Total""","""Variación""",1
"""EMF""","""AMCC""","""04""","""Hogares""","""Variación""",1
"""EMF""","""AMCC""","""04""","""Empresas Priva…","""Saldo""",1
"""EMF""","""AMCC""","""04""","""Empresas Priva…","""Variación""",1


Para ver todas las variables:


In [157]:
res_nivel_7(
    "EMF","AMCC",
    "Niv_Gr_3","04",
    "Id","Niv_Descr_1","Niv_Descr_3","Niv_Descr_4")

<string>:29: FutureWarning:

Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.



Niv_Gr_1,Niv_Gr_2,Niv_Gr_3,Id,Niv_Descr_1,Niv_Descr_3,Niv_Descr_4,N_Variables
str,str,str,i64,str,str,str,u32
"""EMF""","""AMCC""","""04""",645,"""Crédito Otras …","""Total""","""Saldo""",1
"""EMF""","""AMCC""","""04""",648,"""Crédito Otras …","""Total""","""Variación""",1
"""EMF""","""AMCC""","""04""",643,"""Crédito Otras …","""Empresas Priva…","""Saldo""",1
"""EMF""","""AMCC""","""04""",646,"""Crédito Otras …","""Empresas Priva…","""Variación""",1
"""EMF""","""AMCC""","""04""",647,"""Crédito Otras …","""Hogares""","""Variación""",1
"""EMF""","""AMCC""","""04""",644,"""Crédito Otras …","""Hogares""","""Saldo""",1


Resultados de una sola variable:


In [158]:
Id = 648
df = get_df(Id)
# get_plot(Id)
df

<string>:29: FutureWarning:

Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.



<string>:29: FutureWarning:

Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.



Fecha,Crédito Otras Sociedades de Depósito-VariaciónCréditodelasOSDTotal-Total-Variación
date,f64
2010-01-01,2.4085
2010-02-01,2.1975
2010-03-01,2.3382
2010-04-01,3.0188
2010-05-01,3.1448
…,…
2023-11-01,18.591
2023-12-01,18.7543
2024-01-01,18.5811


##### 05 Saldo de Préstamos (Millones de Lempiras)


In [159]:
res_nivel_4(
    "EMF","AMCC",
    "Niv_Gr_3","05",
    "Niv_Descr_1")

<string>:29: FutureWarning:

Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.



Niv_Gr_1,Niv_Gr_2,Niv_Gr_3,Niv_Descr_1,N_Variables
str,str,str,str,u32
"""EMF""","""AMCC""","""05""","""Saldo de prést…",14


* Variación
* Saldo

De los siguientes sectores:

* Agropecuaria
* Comercio
* Consumo
* Industria
* Propiedad Raí­z
* Servicios
* Total

Para ver todas las variables:


In [160]:
res_nivel_7(
    "EMF","AMCC",
    "Niv_Gr_3","05",
    "Id","Niv_Descr_1","Niv_Descr_3","Niv_Descr_4").sort("Niv_Descr_4")

<string>:29: FutureWarning:

Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.



Niv_Gr_1,Niv_Gr_2,Niv_Gr_3,Id,Niv_Descr_1,Niv_Descr_3,Niv_Descr_4,N_Variables
str,str,str,i64,str,str,str,u32
"""EMF""","""AMCC""","""05""",650,"""Saldo de prést…","""Comercio""","""Saldo""",1
"""EMF""","""AMCC""","""05""",655,"""Saldo de prést…","""Total""","""Saldo""",1
"""EMF""","""AMCC""","""05""",651,"""Saldo de prést…","""Consumo""","""Saldo""",1
"""EMF""","""AMCC""","""05""",653,"""Saldo de prést…","""PropiedadRaíz""","""Saldo""",1
"""EMF""","""AMCC""","""05""",654,"""Saldo de prést…","""Servicios""","""Saldo""",1
…,…,…,…,…,…,…,…
"""EMF""","""AMCC""","""05""",660,"""Saldo de prést…","""VariaciónPropi…","""Variación""",1
"""EMF""","""AMCC""","""05""",659,"""Saldo de prést…","""VariaciónIndus…","""Variación""",1
"""EMF""","""AMCC""","""05""",661,"""Saldo de prést…","""VariaciónServi…","""Variación""",1


Resultados de una sola variable:


In [161]:
Id = 650
df = get_df(Id)
# get_plot(Id)
df

<string>:29: FutureWarning:

Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.



<string>:29: FutureWarning:

Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.



Fecha,Saldo de préstamos-Saldos en millones de lempiras-Comercio-Saldo
date,f64
2017-01-01,40207.1
2017-02-01,40690.6
2017-03-01,40997.5
2017-04-01,40674.4
2017-05-01,39759.4
…,…
2023-08-01,90128.6
2023-09-01,90088.3
2023-10-01,90823.6


##### 06 Préstamos Nuevos (Flujos acumulados en millones de lempiras)


In [162]:
res_nivel_4(
    "EMF","AMCC",
    "Niv_Gr_3","06",
    "Niv_Descr_1")

<string>:29: FutureWarning:

Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.



Niv_Gr_1,Niv_Gr_2,Niv_Gr_3,Niv_Descr_1,N_Variables
str,str,str,str,u32
"""EMF""","""AMCC""","""06""","""Préstamos nuev…",14


* Variación
* Saldo

De los siguientes sectores:

* Agropecuaria
* Comercio
* Consumo
* Industria
* Propiedad Raí­z
* Servicios
* Total


In [163]:
res_nivel_5(
    "EMF","AMCC",
    "Niv_Gr_3","06",
    "Niv_Descr_1","Niv_Descr_4")

<string>:29: FutureWarning:

Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.



Niv_Gr_1,Niv_Gr_2,Niv_Gr_3,Niv_Descr_1,Niv_Descr_4,N_Variables
str,str,str,str,str,u32
"""EMF""","""AMCC""","""06""","""Préstamos nuev…","""Saldo""",7
"""EMF""","""AMCC""","""06""","""Préstamos nuev…","""Variación""",7


Para ver todas las variables:


In [164]:
res_nivel_7(
    "EMF","AMCC",
    "Niv_Gr_3","06",
    "Id","Niv_Descr_1","Niv_Descr_3","Niv_Descr_4").sort("Niv_Descr_4")

<string>:29: FutureWarning:

Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.



Niv_Gr_1,Niv_Gr_2,Niv_Gr_3,Id,Niv_Descr_1,Niv_Descr_3,Niv_Descr_4,N_Variables
str,str,str,i64,str,str,str,u32
"""EMF""","""AMCC""","""06""",663,"""Préstamos nuev…","""Agropecuaria""","""Saldo""",1
"""EMF""","""AMCC""","""06""",665,"""Préstamos nuev…","""Consumo""","""Saldo""",1
"""EMF""","""AMCC""","""06""",668,"""Préstamos nuev…","""Servicios""","""Saldo""",1
"""EMF""","""AMCC""","""06""",666,"""Préstamos nuev…","""Industria""","""Saldo""",1
"""EMF""","""AMCC""","""06""",667,"""Préstamos nuev…","""PropiedadRaíz""","""Saldo""",1
…,…,…,…,…,…,…,…
"""EMF""","""AMCC""","""06""",676,"""Préstamos nuev…","""VariaciónTotal…","""Variación""",1
"""EMF""","""AMCC""","""06""",672,"""Préstamos nuev…","""VariaciónConsu…","""Variación""",1
"""EMF""","""AMCC""","""06""",673,"""Préstamos nuev…","""VariaciónIndus…","""Variación""",1


Resultados de una sola variable:


In [165]:
Id = 663
df = get_df(Id)
# get_plot(Id)
df

<string>:29: FutureWarning:

Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.



<string>:29: FutureWarning:

Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.



Fecha,Préstamos nuevos-Flujos acumulados en millones de lempiras-Agropecuaria-Saldo
date,f64
2017-01-01,1326.8
2017-02-01,1859.6
2017-03-01,2709.4
2017-04-01,3727.6
2017-05-01,4787.8
…,…
2023-08-01,11386.5
2023-09-01,12141.2
2023-10-01,13187.9


#### TI = Tasas de Interés


In [166]:
res_nivel_3("EMF","TI","Niv_Gr_3").sort("Niv_Gr_3")

3 subgrupos
38 variables


<string>:29: FutureWarning:

Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.



Niv_Gr_1,Niv_Gr_2,Niv_Gr_3,N_Variables
str,str,str,u32
"""EMF""","""TI""","""01""",8
"""EMF""","""TI""","""02""",20
"""EMF""","""TI""","""03""",10


##### 01 Tasas de Interés, Activa y Pasiva, por Moneda

Por Tipo:

* Activa Préstamos 
* Pasiva Ahorros 
* Pasiva Cheques 
* Pasiva Plazo 


In [167]:
res_nivel_4(
    "EMF","TI",
    "Niv_Gr_3","01",
    "Niv_Descr_5")

<string>:29: FutureWarning:

Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.



Niv_Gr_1,Niv_Gr_2,Niv_Gr_3,Niv_Descr_5,N_Variables
str,str,str,str,u32
"""EMF""","""TI""","""01""",""" Pasiva Ahorro…",2
"""EMF""","""TI""","""01""",""" Pasiva Plazo …",2
"""EMF""","""TI""","""01""",""" Pasiva Cheque…",2
"""EMF""","""TI""","""01""",""" Activa Présta…",2


Por Moneda:

* Moneda Nacional
* Moneda Extranjera

Para ver todas las variables:


In [168]:
res_nivel_7(
    "EMF","TI",
    "Niv_Gr_3","01",
    "Id","Nombre","Niv_Descr_5","Niv_Descr_6")

<string>:29: FutureWarning:

Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.



Niv_Gr_1,Niv_Gr_2,Niv_Gr_3,Id,Nombre,Niv_Descr_5,Niv_Descr_6,N_Variables
str,str,str,i64,str,str,str,u32
"""EMF""","""TI""","""01""",587,"""EMF-TI-01-3""",""" Pasiva Plazo …",""" Moneda Extran…",1
"""EMF""","""TI""","""01""",585,"""EMF-TI-01-7""",""" Pasiva Cheque…",""" Moneda Extran…",1
"""EMF""","""TI""","""01""",582,"""EMF-TI-01-6""",""" Pasiva Ahorro…",""" Moneda Nacion…",1
"""EMF""","""TI""","""01""",583,"""EMF-TI-01-5""",""" Pasiva Ahorro…",""" Moneda Extran…",1
"""EMF""","""TI""","""01""",586,"""EMF-TI-01-4""",""" Pasiva Plazo …",""" Moneda Nacion…",1
"""EMF""","""TI""","""01""",584,"""EMF-TI-01-8""",""" Pasiva Cheque…",""" Moneda Nacion…",1
"""EMF""","""TI""","""01""",580,"""EMF-TI-01-2""",""" Activa Présta…",""" Moneda Nacion…",1
"""EMF""","""TI""","""01""",581,"""EMF-TI-01-1""",""" Activa Présta…",""" Moneda Extran…",1


Resultados de una sola variable:


In [169]:
Id = 587
df = get_df(Id)
# get_plot(Id)
df

<string>:29: FutureWarning:

Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.



<string>:29: FutureWarning:

Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.



Fecha,EMF-TI-01 - PasivaaPlazoMonedaExtranjera - Pasiva Plazo - Moneda Extranjera
date,f64
1998-01-01,9.3
1998-02-01,8.76
1998-03-01,9.33
1998-04-01,9.54
1998-05-01,9.38
…,…
2023-11-01,4.93
2023-12-01,5.12
2024-01-01,5.29


##### 02 Por Actividad


In [170]:
res_nivel_4(
    "EMF","TI",
    "Niv_Gr_3","02",
    "Niv_Descr_5")

<string>:29: FutureWarning:

Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.



Niv_Gr_1,Niv_Gr_2,Niv_Gr_3,Niv_Descr_5,N_Variables
str,str,str,str,u32
"""EMF""","""TI""","""02""",""" Industrias Ma…",2
"""EMF""","""TI""","""02""",""" Actividades A…",2
"""EMF""","""TI""","""02""",""" Consumo""",2
"""EMF""","""TI""","""02""",""" Transporte, A…",2
"""EMF""","""TI""","""02""",""" Explotación d…",2
"""EMF""","""TI""","""02""",""" Construcción""",2
"""EMF""","""TI""","""02""",""" Vivienda""",2
"""EMF""","""TI""","""02""",""" Agricultura, …",2
"""EMF""","""TI""","""02""",""" Otros Sectore…",2


Por Moneda:

* Moneda Nacional
* Moneda Extranjera


In [171]:
res_nivel_5(
    "EMF","TI",
    "Niv_Gr_3","02",
    "Niv_Descr_1","Niv_Descr_4")

<string>:29: FutureWarning:

Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.



Niv_Gr_1,Niv_Gr_2,Niv_Gr_3,Niv_Descr_1,Niv_Descr_4,N_Variables
str,str,str,str,str,u32
"""EMF""","""TI""","""02""","""EMF""",""" Nacional """,10
"""EMF""","""TI""","""02""","""EMF""",""" Extranjera """,10


Para ver todas las variables:


In [172]:
res_nivel_7(
    "EMF","TI",
    "Niv_Gr_3","02",
    "Id","Nombre","Niv_Descr_5","Niv_Descr_4").sort("Niv_Descr_4")

<string>:29: FutureWarning:

Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.



Niv_Gr_1,Niv_Gr_2,Niv_Gr_3,Id,Nombre,Niv_Descr_5,Niv_Descr_4,N_Variables
str,str,str,i64,str,str,str,u32
"""EMF""","""TI""","""02""",600,"""EMF-TI-02-5""",""" Industrias Ma…",""" Extranjera """,1
"""EMF""","""TI""","""02""",605,"""EMF-TI-02-10""",""" Actividades A…",""" Extranjera """,1
"""EMF""","""TI""","""02""",598,"""EMF-TI-02-3""",""" Agricultura, …",""" Extranjera """,1
"""EMF""","""TI""","""02""",602,"""EMF-TI-02-7""",""" Consumo""",""" Extranjera """,1
"""EMF""","""TI""","""02""",607,"""EMF-TI-02-13""",""" Otros Sectore…",""" Extranjera """,1
…,…,…,…,…,…,…,…
"""EMF""","""TI""","""02""",591,"""EMF-TI-02-15""",""" Construcción""",""" Nacional """,1
"""EMF""","""TI""","""02""",589,"""EMF-TI-02-12""",""" Explotación d…",""" Nacional """,1
"""EMF""","""TI""","""02""",592,"""EMF-TI-02-16""",""" Consumo""",""" Nacional """,1


Resultados de una sola variable:


In [173]:
Id = 605
df = get_df(Id)
# get_plot(Id)
df

<string>:29: FutureWarning:

Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.



<string>:29: FutureWarning:

Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.



Fecha,EMF-TI-02 - Extranjera - Actividades Alojamiento y Servicios Comida
date,f64
2019-01-01,6.2445
2019-02-01,6.2271
2019-03-01,6.1944
2019-04-01,6.3569
2019-05-01,6.3522
…,…
2023-10-01,6.29
2023-11-01,7.01
2023-12-01,6.3507


##### 03 Tasas de Interés Anuales

* Tasas de Interés Anuales Mí­nimas y Máximas del Sistema Bancario Comercial
* Tasas de Interés Anuales Promedio Ponderadas del Sistema Financiero


In [174]:
res_nivel_4(
    "EMF","TI",
    "Niv_Gr_3","03",
    "Niv_Descr_6")

<string>:29: FutureWarning:

Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.



Niv_Gr_1,Niv_Gr_2,Niv_Gr_3,Niv_Descr_6,N_Variables
str,str,str,str,u32
"""EMF""","""TI""","""03""",""" Tasas de Inte…",8
"""EMF""","""TI""","""03""",""" Tasas de Inte…",2


* Activa
* Pasiva

Por Unidad de Medida:

* Máxima
* Mínima
* Promedio

Por Moneda:

* Nacional
* Extranjera

Para ver todas las variables:


In [175]:
res_nivel_7(
    "EMF","TI",
    "Niv_Gr_3","03",
    "Id","Nombre","Niv_Descr_6","Niv_Descr_5").sort("Niv_Descr_6")

<string>:29: FutureWarning:

Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.



Niv_Gr_1,Niv_Gr_2,Niv_Gr_3,Id,Nombre,Niv_Descr_6,Niv_Descr_5,N_Variables
str,str,str,i64,str,str,str,u32
"""EMF""","""TI""","""03""",695,"""EMF-TI-03-7""",""" Tasas de Inte…","""Pasiva Mínima …",1
"""EMF""","""TI""","""03""",690,"""EMF-TI-03-2""",""" Tasas de Inte…","""Activa Máxima …",1
"""EMF""","""TI""","""03""",693,"""EMF-TI-03-5""",""" Tasas de Inte…","""Activa Mínima …",1
"""EMF""","""TI""","""03""",694,"""EMF-TI-03-6""",""" Tasas de Inte…","""Activa Máxima …",1
"""EMF""","""TI""","""03""",696,"""EMF-TI-03-8""",""" Tasas de Inte…","""Pasiva Máxima …",1
"""EMF""","""TI""","""03""",691,"""EMF-TI-03-3""",""" Tasas de Inte…","""Pasiva Mínima …",1
"""EMF""","""TI""","""03""",689,"""EMF-TI-03-1""",""" Tasas de Inte…","""Activa Mínima …",1
"""EMF""","""TI""","""03""",692,"""EMF-TI-03-4""",""" Tasas de Inte…","""Pasiva Máxima …",1
"""EMF""","""TI""","""03""",698,"""EMF-TI-03-10""",""" Tasas de Inte…","""Activa Promedi…",1


Resultados de una sola variable:


In [176]:
Id = 689
df = get_df(Id)
# get_plot(Id)
df

<string>:29: FutureWarning:

Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.



<string>:29: FutureWarning:

Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.



Fecha,EMF-TI-03-1-Activa Mínima en Moneda Nacional - Tasas de Interés Anuales Mínimas y Máximas del Sistema Bancario Comercial-C1
date,f64
2018-01-01,9.17
2018-02-01,9.75
2018-03-01,12.86
2018-04-01,12.58
2018-05-01,12.71
…,…
2023-09-01,4.94
2023-10-01,7.63
2023-11-01,6.76


#### PFS = Panorama de las Sociedades Financieras


In [177]:
res_nivel_3("EMF","PFS","Niv_Gr_3").sort("Niv_Gr_3")

2 subgrupos
12 variables


<string>:29: FutureWarning:

Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.



Niv_Gr_1,Niv_Gr_2,Niv_Gr_3,N_Variables
str,str,str,u32
"""EMF""","""PFS""","""01""",3
"""EMF""","""PFS""","""02""",9


##### 01 Bancos Comerciales, Activos y Pasivos Externos

Para ver todas las variables:


In [178]:
res_nivel_7(
    "EMF","PFS",
    "Niv_Gr_3","01",
    "Id","Nombre","Niv_Descr_2","Niv_Descr_1").sort("Nombre")

<string>:29: FutureWarning:

Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.



Niv_Gr_1,Niv_Gr_2,Niv_Gr_3,Id,Nombre,Niv_Descr_2,Niv_Descr_1,N_Variables
str,str,str,i64,str,str,str,u32
"""EMF""","""PFS""","""01""",677,"""EMF-PFS-01-1""","""Bancos Comerci…","""Activos Extern…",1
"""EMF""","""PFS""","""01""",678,"""EMF-PFS-01-2""","""Bancos Comerci…","""Activos Extern…",1
"""EMF""","""PFS""","""01""",679,"""EMF-PFS-01-3""","""Bancos Comerci…","""Pasivos Extern…",1


Resultados de una sola variable:


In [179]:
Id = 677
df = get_df(Id)
# get_plot(Id)
df

<string>:29: FutureWarning:

Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.



<string>:29: FutureWarning:

Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.



Fecha,Activos Externos Netos-Bancos Comerciales
date,f64
2009-01-01,-1855.5907
2009-02-01,-723.8547
2009-03-01,-862.9082
2009-04-01,80.2026
2009-05-01,792.7156
…,…
2023-11-01,-32943.865
2023-12-01,-36239.6994
2024-01-01,-37030.809


##### 02 Activos y Pasivos Externos


In [180]:
res_nivel_4(
    "EMF","PFS",
    "Niv_Gr_3","02",
    "Niv_Descr_1")

<string>:29: FutureWarning:

Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.



Niv_Gr_1,Niv_Gr_2,Niv_Gr_3,Niv_Descr_1,N_Variables
str,str,str,str,u32
"""EMF""","""PFS""","""02""","""Activos Extern…",3
"""EMF""","""PFS""","""02""","""Activos Extern…",3
"""EMF""","""PFS""","""02""","""Pasivos Extern…",3


In [181]:
res_nivel_4(
    "EMF","PFS",
    "Niv_Gr_3","02",
    "Niv_Descr_2")

<string>:29: FutureWarning:

Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.



Niv_Gr_1,Niv_Gr_2,Niv_Gr_3,Niv_Descr_2,N_Variables
str,str,str,str,u32
"""EMF""","""PFS""","""02""","""Banco Central …",3
"""EMF""","""PFS""","""02""","""Otras Sociedad…",3
"""EMF""","""PFS""","""02""","""Sociedades de …",3


Para ver todas las variables:


In [182]:
res_nivel_7(
    "EMF","PFS",
    "Niv_Gr_3","02",
    "Id","Nombre","Niv_Descr_1","Niv_Descr_2")

<string>:29: FutureWarning:

Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.



Niv_Gr_1,Niv_Gr_2,Niv_Gr_3,Id,Nombre,Niv_Descr_1,Niv_Descr_2,N_Variables
str,str,str,i64,str,str,str,u32
"""EMF""","""PFS""","""02""",680,"""EMF-PFS-02-1""","""Activos Extern…","""Banco Central …",1
"""EMF""","""PFS""","""02""",684,"""EMF-PFS-02-5""","""Activos Extern…","""Otras Sociedad…",1
"""EMF""","""PFS""","""02""",687,"""EMF-PFS-02-8""","""Pasivos Extern…","""Otras Sociedad…",1
"""EMF""","""PFS""","""02""",681,"""EMF-PFS-02-2""","""Activos Extern…","""Otras Sociedad…",1
"""EMF""","""PFS""","""02""",686,"""EMF-PFS-02-7""","""Pasivos Extern…","""Banco Central …",1
"""EMF""","""PFS""","""02""",682,"""EMF-PFS-02-3""","""Activos Extern…","""Sociedades de …",1
"""EMF""","""PFS""","""02""",685,"""EMF-PFS-02-6""","""Activos Extern…","""Sociedades de …",1
"""EMF""","""PFS""","""02""",683,"""EMF-PFS-02-4""","""Activos Extern…","""Banco Central …",1
"""EMF""","""PFS""","""02""",688,"""EMF-PFS-02-9""","""Pasivos Extern…","""Sociedades de …",1


Resultados de una sola variable:


In [183]:
Id = 687
df = get_df(Id)
# get_plot(Id)
df

<string>:29: FutureWarning:

Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.



<string>:29: FutureWarning:

Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.



Fecha,Pasivos Externos-Otras Sociedades de Depósito
date,f64
2009-01-01,15258.1141
2009-02-01,15135.3623
2009-03-01,14597.5965
2009-04-01,14237.5792
2009-05-01,13918.5834
…,…
2023-11-01,52769.2959
2023-12-01,56670.0158
2024-01-01,56699.17


#### EBM = Emisión y Base Monetaria


In [184]:
res_nivel_3("EMF","EBM","Niv_Gr_3").sort("Niv_Gr_3")

1 subgrupos
6 variables


<string>:29: FutureWarning:

Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.



Niv_Gr_1,Niv_Gr_2,Niv_Gr_3,N_Variables
str,str,str,u32
"""EMF""","""EBM""","""01""",6


In [185]:
res_nivel_4(
    "EMF","EBM",
    "Niv_Gr_3","01",
    "Niv_Descr_4")

<string>:29: FutureWarning:

Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.



Niv_Gr_1,Niv_Gr_2,Niv_Gr_3,Niv_Descr_4,N_Variables
str,str,str,str,u32
"""EMF""","""EBM""","""01""","""Emision Moneta…",2
"""EMF""","""EBM""","""01""","""Base Monetaria…",2
"""EMF""","""EBM""","""01""","""Base Monetaria…",2


In [186]:
res_nivel_5(
    "EMF","EBM",
    "Niv_Gr_3","01",
    "Niv_Descr_4","Niv_Descr_5")

<string>:29: FutureWarning:

Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.



Niv_Gr_1,Niv_Gr_2,Niv_Gr_3,Niv_Descr_4,Niv_Descr_5,N_Variables
str,str,str,str,str,u32
"""EMF""","""EBM""","""01""","""Base Monetaria…","""Saldo""",1
"""EMF""","""EBM""","""01""","""Emision Moneta…","""Variación""",1
"""EMF""","""EBM""","""01""","""Emision Moneta…","""Saldo""",1
"""EMF""","""EBM""","""01""","""Base Monetaria…","""Variación""",1
"""EMF""","""EBM""","""01""","""Base Monetaria…","""Variación""",1
"""EMF""","""EBM""","""01""","""Base Monetaria…","""Saldo""",1


Para ver todas las variables:


In [187]:
res_nivel_7(
    "EMF","EBM",
    "Niv_Gr_3","01",
    "Id","Nombre","Niv_Descr_4","Niv_Descr_5")

<string>:29: FutureWarning:

Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.



Niv_Gr_1,Niv_Gr_2,Niv_Gr_3,Id,Nombre,Niv_Descr_4,Niv_Descr_5,N_Variables
str,str,str,i64,str,str,str,u32
"""EMF""","""EBM""","""01""",522,"""EMF-EBM-01-3""","""Base Monetaria…","""Saldo""",1
"""EMF""","""EBM""","""01""",518,"""EMF-EBM-01-1""","""Emision Moneta…","""Saldo""",1
"""EMF""","""EBM""","""01""",519,"""EMF-EBM-01-4""","""Emision Moneta…","""Variación""",1
"""EMF""","""EBM""","""01""",521,"""EMF-EBM-01-5""","""Base Monetaria…","""Variación""",1
"""EMF""","""EBM""","""01""",523,"""EMF-EBM-01-6""","""Base Monetaria…","""Variación""",1
"""EMF""","""EBM""","""01""",520,"""EMF-EBM-01-2""","""Base Monetaria…","""Saldo""",1


Resultados de una sola variable:


In [188]:
Id = 523
df = get_df(Id)
# get_plot(Id)
df

<string>:29: FutureWarning:

Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.



<string>:29: FutureWarning:

Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.



Fecha,EMF-EBM-01-Base Monetaria Amplia-Variación
date,f64
2009-12-01,7.7574
2010-01-01,6.7993
2010-02-01,9.6529
2010-03-01,12.938
2010-04-01,7.1
…,…
2023-11-01,-1.2691
2023-12-01,-0.3203
2024-01-01,-0.8613


### ESP = Estadísticas de Sistema de Pagos


In [189]:
res_nivel_2("ESP")

11519 variables
54 grupos
2 variables
2 grupos
2 subgrupos


<string>:29: FutureWarning:

Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.



Niv_Gr_1,Niv_Gr_2,N_Variables,N_Niv_Gr_2
str,str,u32,u32
"""ESP""","""CCECH""",1,1
"""ESP""","""ACH""",1,1


#### ACH = Operaciones de Crédito Compensadas en Ceproban y Liquidadas en BCH


In [190]:
res_nivel_3("ESP","ACH","Niv_Gr_3").sort("Niv_Gr_3")

1 subgrupos
4 variables


<string>:29: FutureWarning:

Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.



Niv_Gr_1,Niv_Gr_2,Niv_Gr_3,N_Variables
str,str,str,u32
"""ESP""","""ACH""","""01""",4


In [191]:
res_nivel_4(
    "ESP","ACH",
    "Niv_Gr_3","01",
    "Niv_Descr_5")

<string>:29: FutureWarning:

Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.



Niv_Gr_1,Niv_Gr_2,Niv_Gr_3,Niv_Descr_5,N_Variables
str,str,str,str,u32
"""ESP""","""ACH""","""01""","""Cantidad""",2
"""ESP""","""ACH""","""01""","""Monto""",2


In [192]:
res_nivel_4(
    "ESP","ACH",
    "Niv_Gr_3","01",
    "Niv_Descr_6")

<string>:29: FutureWarning:

Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.



Niv_Gr_1,Niv_Gr_2,Niv_Gr_3,Niv_Descr_6,N_Variables
str,str,str,str,u32
"""ESP""","""ACH""","""01""","""Dólares""",2
"""ESP""","""ACH""","""01""","""Lempiras""",2


Para ver todas las variables:


In [193]:
res_nivel_7(
    "ESP","ACH",
    "Niv_Gr_3","01",
    "Id","Nombre","Niv_Descr_5","Niv_Descr_6").sort("Nombre")

<string>:29: FutureWarning:

Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.



Niv_Gr_1,Niv_Gr_2,Niv_Gr_3,Id,Nombre,Niv_Descr_5,Niv_Descr_6,N_Variables
str,str,str,i64,str,str,str,u32
"""ESP""","""ACH""","""01""",611,"""ESP-ACH-01-1""","""Cantidad""","""Dólares""",1
"""ESP""","""ACH""","""01""",612,"""ESP-ACH-01-2""","""Cantidad""","""Lempiras""",1
"""ESP""","""ACH""","""01""",613,"""ESP-ACH-01-3""","""Monto""","""Dólares""",1
"""ESP""","""ACH""","""01""",614,"""ESP-ACH-01-4""","""Monto""","""Lempiras""",1


Resultados de una sola variable:


In [194]:
Id = 611
df = get_df(Id)
# get_plot(Id)
df

<string>:29: FutureWarning:

Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.



<string>:29: FutureWarning:

Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.



Fecha,ESP-ACH-01-1-Cantidad-Dólares
date,i64
2010-01-01,3
2010-02-01,9
2010-03-01,34
2010-04-01,54
2010-05-01,67
…,…
2023-10-01,31605
2023-11-01,31941
2023-12-01,31208


#### CCECH = Cheques Compensados en Ceproban y Liquidados en BCH


In [195]:
res_nivel_3("ESP","CCECH","Niv_Gr_3")

1 subgrupos
4 variables


<string>:29: FutureWarning:

Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.



Niv_Gr_1,Niv_Gr_2,Niv_Gr_3,N_Variables
str,str,str,u32
"""ESP""","""CCECH""","""01""",4


In [196]:
res_nivel_4(
    "ESP","CCECH",
    "Niv_Gr_3","01",
    "Niv_Descr_5")

<string>:29: FutureWarning:

Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.



Niv_Gr_1,Niv_Gr_2,Niv_Gr_3,Niv_Descr_5,N_Variables
str,str,str,str,u32
"""ESP""","""CCECH""","""01""","""Cantidad""",2
"""ESP""","""CCECH""","""01""","""Monto""",2


In [197]:
res_nivel_4(
    "ESP","CCECH",
    "Niv_Gr_3","01",
    "Niv_Descr_6")

<string>:29: FutureWarning:

Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.



Niv_Gr_1,Niv_Gr_2,Niv_Gr_3,Niv_Descr_6,N_Variables
str,str,str,str,u32
"""ESP""","""CCECH""","""01""","""Lempiras""",2
"""ESP""","""CCECH""","""01""","""Dólares""",2


Para ver todas las variables:


In [198]:
res_nivel_7(
    "ESP","CCECH",
    "Niv_Gr_3","01",
    "Id","Nombre","Niv_Descr_5","Niv_Descr_6")

<string>:29: FutureWarning:

Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.



Niv_Gr_1,Niv_Gr_2,Niv_Gr_3,Id,Nombre,Niv_Descr_5,Niv_Descr_6,N_Variables
str,str,str,i64,str,str,str,u32
"""ESP""","""CCECH""","""01""",617,"""ESP-CCECH-01-3…","""Monto""","""Dólares""",1
"""ESP""","""CCECH""","""01""",618,"""ESP-CCECH-01-4…","""Monto""","""Lempiras""",1
"""ESP""","""CCECH""","""01""",615,"""ESP-CCECH-01-1…","""Cantidad""","""Dólares""",1
"""ESP""","""CCECH""","""01""",616,"""ESP-CCECH-01-2…","""Cantidad""","""Lempiras""",1


Resultados de una sola variable:


In [199]:
Id = 615
df = get_df(Id)
# get_plot(Id)
df

<string>:29: FutureWarning:

Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.



<string>:29: FutureWarning:

Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.



Fecha,ESP-CCECH-01-1-Cantidad-Dólares
date,i64
2010-01-01,9344
2010-02-01,10124
2010-03-01,11828
2010-04-01,10274
2010-05-01,11172
…,…
2023-10-01,1628
2023-11-01,1588
2023-12-01,1472


### EM = Estadísticas Monetarias (TPM y RIN)


In [200]:
res_nivel_2("EM")

11519 variables
54 grupos
2 variables
2 grupos
2 subgrupos


<string>:29: FutureWarning:

Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.



Niv_Gr_1,Niv_Gr_2,N_Variables,N_Niv_Gr_2
str,str,u32,u32
"""EM""","""RIN""",1,1
"""EM""","""TPM""",1,1


#### RIN = Reservas Internacionales Netas


In [201]:
res_nivel_3("EM","RIN","Niv_Gr_3").sort("Niv_Gr_3")

1 subgrupos
10 variables


<string>:29: FutureWarning:

Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.



Niv_Gr_1,Niv_Gr_2,Niv_Gr_3,N_Variables
str,str,str,u32
"""EM""","""RIN""","""01""",10


* Saldo
* Variación

Variables:

* Reservas Internacionales Netas
* Activos de Reserva Oficial


In [202]:
res_nivel_4(
    "EM","RIN",
    "Niv_Gr_3","01",
    "Niv_Descr_4")

<string>:29: FutureWarning:

Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.



Niv_Gr_1,Niv_Gr_2,Niv_Gr_3,Niv_Descr_4,N_Variables
str,str,str,str,u32
"""EM""","""RIN""","""01""",""" Activos de Re…",5
"""EM""","""RIN""","""01""",""" Reservas Inte…",5


In [203]:
res_nivel_4(
    "EM","RIN",
    "Niv_Gr_3","01",
    "Niv_Descr_5")

<string>:29: FutureWarning:

Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.



Niv_Gr_1,Niv_Gr_2,Niv_Gr_3,Niv_Descr_5,N_Variables
str,str,str,str,u32
"""EM""","""RIN""","""01""",""" Variación Abs…",2
"""EM""","""RIN""","""01""",""" Variación Abs…",2
"""EM""","""RIN""","""01""",""" Saldo""",2
"""EM""","""RIN""","""01""",""" Variación Abs…",2
"""EM""","""RIN""","""01""",""" Variación Int…",2


Para ver todas las variables:


In [204]:
res_nivel_7(
    "EM","RIN",
    "Niv_Gr_3","01",
    "Id","Nombre","Niv_Descr_4","Niv_Descr_5")

<string>:29: FutureWarning:

Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.



Niv_Gr_1,Niv_Gr_2,Niv_Gr_3,Id,Nombre,Niv_Descr_4,Niv_Descr_5,N_Variables
str,str,str,i64,str,str,str,u32
"""EM""","""RIN""","""01""",524,"""EM-RIN-01-1""",""" Reservas Inte…",""" Saldo""",1
"""EM""","""RIN""","""01""",533,"""EM-RIN-01-2""",""" Activos de Re…",""" Variación Int…",1
"""EM""","""RIN""","""01""",527,"""EM-RIN-01-5""",""" Reservas Inte…",""" Variación Abs…",1
"""EM""","""RIN""","""01""",531,"""EM-RIN-01-9""",""" Activos de Re…",""" Variación Abs…",1
"""EM""","""RIN""","""01""",532,"""EM-RIN-01-10""",""" Activos de Re…",""" Variación Abs…",1
"""EM""","""RIN""","""01""",526,"""EM-RIN-01-4""",""" Reservas Inte…",""" Variación Abs…",1
"""EM""","""RIN""","""01""",525,"""EM-RIN-01-3""",""" Reservas Inte…",""" Variación Abs…",1
"""EM""","""RIN""","""01""",530,"""EM-RIN-01-8""",""" Activos de Re…",""" Variación Abs…",1
"""EM""","""RIN""","""01""",528,"""EM-RIN-01-6""",""" Reservas Inte…",""" Variación Int…",1


Resultados de una sola variable:


In [205]:
Id = 532
df = get_df(Id)
# get_plot(Id)
df

<string>:29: FutureWarning:

Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.



<string>:29: FutureWarning:

Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.



Fecha,EM-RIN-01 - Activos de Reserva Oficial - Variación Absoluta Interanual
date,f64
2002-01-01,1547.119
2002-02-01,1570.3292
2002-03-01,1600.73
2002-04-01,1685.4012
2002-05-01,1675.31
…,…
2023-12-01,-863.6644
2024-01-01,-948.2284
2024-02-01,-953.5449


#### TPM = Tasa de Política Monetaria


In [206]:
res_nivel_3("EM","TPM","Niv_Gr_3")

1 subgrupos
2 variables


<string>:29: FutureWarning:

Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.



Niv_Gr_1,Niv_Gr_2,Niv_Gr_3,N_Variables
str,str,str,u32
"""EM""","""TPM""","""01""",2


Para ver todas las variables:


In [207]:
res_nivel_7(
    "EM","TPM",
    "Niv_Gr_3","01",
    "Id","Nombre","Niv_Descr_1","Niv_Descr_2").sort("Niv_Descr_2")

<string>:29: FutureWarning:

Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.



Niv_Gr_1,Niv_Gr_2,Niv_Gr_3,Id,Nombre,Niv_Descr_1,Niv_Descr_2,N_Variables
str,str,str,i64,str,str,str,u32
"""EM""","""TPM""","""01""",699,"""EM-TPM-01-1""","""Tasa Politica …",""" Diario""",1
"""EM""","""TPM""","""01""",700,"""EM-TPM-01-2""","""Tasa Politica …",""" Mensual""",1


Resultados de una sola variable:


In [208]:
Id = 699
df = get_df(Id)
# get_plot(Id)
df

<string>:29: FutureWarning:

Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.



<string>:29: FutureWarning:

Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.



Fecha,Tasa Politica Monetaria - Diario
date,f64
2005-05-31,0.07
2005-06-01,0.07
2005-06-02,0.07
2005-06-03,0.07
2005-06-04,0.07
…,…
2024-05-17,0.03
2024-05-20,0.03
2024-05-21,0.03


### EC = Estadísticas Cambiarias


In [209]:
res_nivel_2("EC")

11519 variables
54 grupos
2 variables
2 grupos
2 subgrupos


<string>:29: FutureWarning:

Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.



Niv_Gr_1,Niv_Gr_2,N_Variables,N_Niv_Gr_2
str,str,u32,u32
"""EC""","""TCR""",1,1
"""EC""","""TCN""",1,1


#### TCR = Tipo de Cambio de Referencia


In [210]:
res_nivel_3("EC","TCR","Niv_Gr_3").sort("Niv_Gr_3")

1 subgrupos
1 variables


<string>:29: FutureWarning:

Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.



Niv_Gr_1,Niv_Gr_2,Niv_Gr_3,N_Variables
str,str,str,u32
"""EC""","""TCR""",null,1


Para ver todas las variables:


In [211]:
# df_all.filter(pl.col("Nombre") == "EC-TCR-01").select(pl.all().is_not_null())
df_all.filter(pl.col("Nombre") == "EC-TCR-01")[:, df_all.select(pl.all().is_not_null().all()).row(0)]

Id,Nombre,Descripcion,Periodicidad,Grupo,CorrelativoGrupo,Niv_Descr_1,Niv_Descr,Niv_Gr_1,Niv_Gr_2,Niv_Gr
i64,str,str,str,str,str,str,i64,str,str,i64
97,"""EC-TCR-01""","""Estadísticas C…","""Diario""","""EC-TCR""","""01""","""Estadísticas C…",2,"""EC""","""TCR""",9


Resultados de una sola variable:


In [212]:
Id = 97
df = get_df(Id)
# get_plot(Id)
df

<string>:29: FutureWarning:

Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.



<string>:29: FutureWarning:

Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.



Fecha,Estadísticas Cambiarias - Tipo de Cambio de Referencia
date,f64
2019-02-20,24.3944
2019-02-21,24.4009
2019-02-22,24.409
2019-02-23,24.4184
2019-02-26,24.4218
…,…
2024-05-29,24.6887
2024-05-30,24.6879
2024-05-31,24.687


#### TCN = Tipo de Cambio Nominal


In [213]:
res_nivel_3("EC","TCN","Niv_Gr_3")

1 subgrupos
2 variables


<string>:29: FutureWarning:

Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.



Niv_Gr_1,Niv_Gr_2,Niv_Gr_3,N_Variables
str,str,str,u32
"""EC""","""TCN""","""01""",2


Para ver todas las variables:


In [214]:
res_nivel_7(
    "EC","TCN",
    "Niv_Gr_3","01",
    "Id","Nombre","Niv_Descr_1","Niv_Descr_2").sort("Niv_Descr_2")

<string>:29: FutureWarning:

Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.



Niv_Gr_1,Niv_Gr_2,Niv_Gr_3,Id,Nombre,Niv_Descr_1,Niv_Descr_2,N_Variables
str,str,str,i64,str,str,str,u32
"""EC""","""TCN""","""01""",619,"""EC-TCN-01-1""","""Tipo de Cambio…",""" Compra""",1
"""EC""","""TCN""","""01""",620,"""EC-TCN-01-2""","""Tipo de Cambio…",""" Venta""",1


Resultados de una sola variable:


In [215]:
Id = 619
df = get_df(Id)
# get_plot(Id)
df

<string>:29: FutureWarning:

Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.



<string>:29: FutureWarning:

Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.



Fecha,Tipo de Cambio Nominal - Compra
date,f64
2000-01-03,14.5
2000-01-04,14.5
2000-01-05,14.51
2000-01-06,14.51
2000-01-07,14.51
…,…
2024-05-29,24.6887
2024-05-30,24.6879
2024-05-31,24.687


### EP = EStadísticas de Precios


In [216]:
res_nivel_2("EP")

11519 variables
54 grupos
1 variables
1 grupos
1 subgrupos


<string>:29: FutureWarning:

Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.



Niv_Gr_1,Niv_Gr_2,N_Variables,N_Niv_Gr_2
str,str,u32,u32
"""EP""","""IPC""",1,1


#### IPC  Índice de Precios al Consumidor


In [217]:
res_nivel_3("EP","IPC","Niv_Gr_3")

1 subgrupos
3 variables


<string>:29: FutureWarning:

Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.



Niv_Gr_1,Niv_Gr_2,Niv_Gr_3,N_Variables
str,str,str,u32
"""EP""","""IPC""","""01""",3


Para ver todas las variables:


In [218]:
res_nivel_7(
    "EP","IPC",
    "Niv_Gr_3","01",
    "Id","Nombre","Niv_Descr_2","Niv_Descr_4").sort("Niv_Descr_2")

<string>:29: FutureWarning:

Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.



Niv_Gr_1,Niv_Gr_2,Niv_Gr_3,Id,Nombre,Niv_Descr_2,Niv_Descr_4,N_Variables
str,str,str,i64,str,str,str,u32
"""EP""","""IPC""","""01""",609,"""EP-IPC-01-2""","""IPC""",""" Variación int…",1
"""EP""","""IPC""","""01""",610,"""EP-IPC-01-3""","""IPC""",""" Variación men…",1
"""EP""","""IPC""","""01""",608,"""EP-IPC-01-1""","""IPC""",""" Índice""",1


Resultados de una sola variable:


In [219]:
Id = 608
df = get_df(Id)
get_plot(Id)
df

<string>:29: FutureWarning:

Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.



<string>:29: FutureWarning:

Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.



<string>:29: FutureWarning:

Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.



<string>:29: FutureWarning:

Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.



Fecha,EP-IPC-01 - Índice
date,f64
1991-01-01,24.09
1991-02-01,24.88
1991-03-01,25.21
1991-04-01,26.04
1991-05-01,26.34
…,…
2024-01-01,426.6
2024-02-01,429.5
2024-03-01,431.6


## Nombre

El campo "Nombre" es un código que contiene hasta 8 niveles de agrupación (tomando en cuenta una separación por "-"), siendo los primeros 7 iguales a las definiciones del campo "Grupo".


In [220]:
df_all.select(
    "Niv_Gr_1","Niv_Gr_2","Nombre",
    "Niv_Descr_1","Niv_Descr_2","Niv_Descr_3","Niv_Descr_4","Niv_Descr_5","Niv_Descr_6","Niv_Descr_7","Niv_Descr_8").sort("Niv_Gr_1")

Niv_Gr_1,Niv_Gr_2,Nombre,Niv_Descr_1,Niv_Descr_2,Niv_Descr_3,Niv_Descr_4,Niv_Descr_5,Niv_Descr_6,Niv_Descr_7,Niv_Descr_8
str,str,str,str,str,str,str,str,str,str,str
"""EC""","""TCR""","""EC-TCR-01""","""Estadísticas C…",""" Tipo de Cambi…",null,null,null,null,null,null
"""EC""","""TCN""","""EC-TCN-01-1""","""Tipo de Cambio…",""" Compra""",null,null,null,null,null,null
"""EC""","""TCN""","""EC-TCN-01-2""","""Tipo de Cambio…",""" Venta""",null,null,null,null,null,null
"""EM""","""RIN""","""EM-RIN-01-1""","""EM""","""RIN""","""01 """,""" Reservas Inte…",""" Saldo""",null,null,null
"""EM""","""RIN""","""EM-RIN-01-3""","""EM""","""RIN""","""01 """,""" Reservas Inte…",""" Variación Abs…",null,null,null
…,…,…,…,…,…,…,…,…,…,…
"""ESR""","""ODA""","""ESR-ODA-01-90""","""OFERTA GLOBAL""","""VALOR AGREGADO…","""INDUSTRIA MANU…","""VARIACIONES RE…",null,null,null,null
"""ESR""","""ODA""","""ESR-ODA-01-91""","""OFERTA GLOBAL""","""VALOR AGREGADO…","""MENOS: SERVICI…","""EN MILLONES DE…",null,null,null,null
"""ESR""","""ODA""","""ESR-ODA-01-92""","""OFERTA GLOBAL""","""VALOR AGREGADO…","""MENOS: SERVICI…","""EN MILLONES DE…",null,null,null,null


## Periodicidad


In [221]:
#| label: tbl-periodicidad
#| tbl-cap: Número de Variables por Periodicidad

# dfmeta = pl.DataFrame(dfmeta)
variable = "Periodicidad"
res = df_all.group_by(variable
    ).agg(
    pl.col("Periodicidad").len().alias("N_Variables"),
    # pl.col("Monto").sum().alias("Millones_USD")
    ).sort("N_Variables",descending=True)
pd.DataFrame(res).to_csv(
        "api/periodicidad.csv",
        sep=";")
print(str(res["N_Variables"].sum()) + " variables")
res

11519 variables


Periodicidad,N_Variables
str,u32
"""Diario""",5579
"""Anual""",4435
"""Trimestral""",1261
"""Mensual""",244


# Obtener Variables por Id

## Una sola variable


In [222]:
# df_all = save_variables()
df_all = save_variables_vscode()
Id = 204
df = get_df(Id)
get_plot(Id)
df.columns
df.write_csv(
    file_path + "/api/data.csv",
    separator=";")
df 

<string>:29: FutureWarning:

Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.



<string>:29: FutureWarning:

Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.



<string>:29: FutureWarning:

Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.



<string>:29: FutureWarning:

Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.



<string>:29: FutureWarning:

Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.



Fecha,ESR-PIBA-03 - Precio - Producto Interno Bruto (pm)
date,f64
2000-01-01,106654.2
2001-01-01,118415.7
2002-01-01,129166.6
2003-01-01,142817.8
2004-01-01,161507.5
…,…
2019-01-01,614917.7
2020-01-01,585733.6
2021-01-01,684203.8


In [223]:
Id = 7
df = get_df(Id)
get_plot(Id)
df.columns
df.write_csv(
    file_path + "/api/data.csv",
    separator=";")
df

<string>:29: FutureWarning:

Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.



<string>:29: FutureWarning:

Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.



<string>:29: FutureWarning:

Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.



<string>:29: FutureWarning:

Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.



Fecha,Serie Desestacionalizada-VALORES CORRIENTES-23-Producto Interno Bruto a precios de mercado
date,f64
2000-03-01,26020.6
2000-06-01,26554.2
2000-09-01,26849.0
2000-12-01,27230.4
2001-03-01,28443.4
…,…
2022-12-01,201538.6
2023-03-01,203879.0
2023-06-01,209919.4


## Varias variables


In [224]:
Id = [1,2,3]
# Id = [6282,204,1]
Id = [608,609,1]

df = get_df(Id[0])
for i in range(1,len(Id)):
    df = df.join(
        get_df(Id[i]), 
        left_on=['Fecha'], 
        right_on=['Fecha'], 
        how='left')
df.write_csv(
    file_path + "/api/data.csv",
    separator=";")
df    

<string>:29: FutureWarning:

Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.



<string>:29: FutureWarning:

Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.



<string>:29: FutureWarning:

Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.



<string>:29: FutureWarning:

Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.



<string>:29: FutureWarning:

Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.



<string>:29: FutureWarning:

Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.



Fecha,EP-IPC-01 - Índice,EP-IPC-01 - Variación interanual,Serie Desestacionalizada-VALORES CONSTANTES-21-Valor Agregado Bruto a precios básicos
date,f64,f64,f64
1991-01-01,24.09,null,null
1991-02-01,24.88,null,null
1991-03-01,25.21,null,null
1991-04-01,26.04,null,null
1991-05-01,26.34,null,null
…,…,…,…
2024-01-01,426.6,5.0,null
2024-02-01,429.5,4.5,null
2024-03-01,431.6,4.76,null
